In [1]:
# --- Parámetros (la CI podrá sobreescribirlos) ---
from pathlib import Path
from datetime import datetime
import pandas as pd
import pytz

# Zona horaria para "hoy"
TZ = pytz.timezone("Europe/Madrid")

# RUN_DATE: si papermill lo inyecta (o ya existe en globals), lo respetamos.
# En caso contrario, usamos la fecha actual en Europe/Madrid.
if "RUN_DATE" in globals() and globals()["RUN_DATE"]:
    _run_date = str(globals()["RUN_DATE"])
else:
    _run_date = datetime.now(TZ).date().strftime("%Y-%m-%d")

# Normalizamos a YYYY-MM-DD
RUN_DATE = pd.to_datetime(_run_date, errors="coerce").date().strftime("%Y-%m-%d")

# SEASON: si no viene dada, la calculamos a partir de RUN_DATE (formato: 2025_26)
if "SEASON" in globals() and globals()["SEASON"]:
    SEASON = globals()["SEASON"]
else:
    _dt = pd.to_datetime(RUN_DATE)
    _y = int(_dt.year) if _dt.month >= 7 else int(_dt.year) - 1
    SEASON = f"{_y}_{(_y+1) % 100:02d}"

# Otros parámetros (solo si no estaban definidos)
MATCHDAY = globals().get("MATCHDAY", None)
MODEL_VERSION = globals().get("MODEL_VERSION", "xgb-local")

# --- Rutas coherentes local/CI ---
ROOT   = Path.cwd()
DATA   = ROOT / "data"
RAW    = DATA / "01_raw"
PROC   = DATA / "02_processed"
FEAT   = DATA / "03_features"
MODELS = DATA / "04_models"
OUT    = ROOT / "outputs"
MANUAL = ROOT / "manual"   # <- carpeta para tus b365_filled_*.csv (la usará la celda grande)

for p in [RAW, PROC, FEAT, MODELS, OUT, MANUAL]:
    p.mkdir(parents=True, exist_ok=True)

print(f"RUN_DATE = {RUN_DATE} | SEASON = {SEASON} | MATCHDAY = {MATCHDAY} | MODEL_VERSION = {MODEL_VERSION}")

RUN_DATE = 2025-12-12 | SEASON = 2025_26 | MATCHDAY = None | MODEL_VERSION = xgb-local


In [2]:
# --- utilidades de IO y dependencias para la celda grande ---
import os, io, glob, requests
import pandas as pd
from pathlib import Path
from itertools import product

def load_raw(name: str):   return pd.read_parquet(RAW / name)
def save_raw(df, name: str):  (RAW).mkdir(exist_ok=True, parents=True); df.to_parquet(RAW / name, index=False)

def load_proc(name: str):  return pd.read_parquet(PROC / name)
def save_proc(df, name: str): (PROC).mkdir(exist_ok=True, parents=True); df.to_parquet(PROC / name, index=False)

def load_feat(name: str):  return pd.read_parquet(FEAT / name)
def save_feat(df, name: str):  (FEAT).mkdir(exist_ok=True, parents=True); df.to_parquet(FEAT / name, index=False)

# Alias útil por si la celda grande quiere detectar la carpeta manual por variable global
MANUAL_DIR = MANUAL

# **EXTRACCIÓN DE LOS DATOS**

In [3]:
# !pip install fake_useragent

In [4]:
# !pip install soccerdata

In [5]:
from itertools import product
from datetime import datetime
from bs4 import BeautifulSoup, Comment
from tqdm import tqdm
from fake_useragent import UserAgent
from datetime import timedelta
from pathlib import Path
from io import StringIO

import pandas as pd
import numpy as np
import soccerdata as sd

import random
import io
import os
import requests
import unicodedata
import re
import time
import glob

[12/12/25 17:11:12] INFO     No custom team name replacements found. You can configure these in       ]8;id=520741;file:///usr/local/lib/python3.12/dist-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=490188;file:///usr/local/lib/python3.12/dist-packages/soccerdata/_config.py#91\91]8;;\
                             /root/soccerdata/config/teamname_replacements.json.                                   

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=523650;file:///usr/local/lib/python3.12/dist-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=284956;file:///usr/local/lib/python3.12/dist-packages/soccerdata/_config.py#197\197]8;;\
                             /root/soccerdata/config/league_dict.json.                                             

# 1. football-data.co.uk

In [7]:
# ╔══════════════════════════════════════════════════════════════════════════╗
# ║  FOOTBALL-DATA.CO.UK • La Liga • MATCHES  (con MANUAL matches + cuotas)  ║
# ╚══════════════════════════════════════════════════════════════════════════╝

RAW_DIR  = RAW / "football-data"
PROC_DIR = PROC
PARQUET_PATH = PROC / "football-data.co.uk_2005_2025.parquet"

RAW_DIR.mkdir(parents=True, exist_ok=True)
PROC_DIR.mkdir(parents=True, exist_ok=True)

DIVISIONS = ["SP1"]
FROZEN = ["B365H", "B365D", "B365A"]

# ----------------- Utilidades -----------------
def _dedup_columns(df: pd.DataFrame, tag: str = "DF") -> pd.DataFrame:
    """Elimina columnas duplicadas conservando la última, y avisa."""
    if df is None or df.empty:
        return df
    dup = df.columns[df.columns.duplicated()].tolist()
    if dup:
        print(f"[{tag}] columnas duplicadas detectadas (se conserva la última): {dup}")
        df = df.loc[:, ~df.columns.duplicated(keep="last")]
    return df

def align_columns_like(df_src: pd.DataFrame, template_cols) -> pd.DataFrame:
    """
    Alinea df_src a las columnas de template_cols sin reindex() y sin fragmentar:
    - Deduplica columnas en origen
    - Crea las faltantes de una sola vez
    - Ordena como el template
    """
    df_src = _dedup_columns(df_src, "ALIGN.SRC").copy()
    tmp = pd.Index(template_cols)
    tmp = tmp[~tmp.duplicated(keep="last")]
    missing = [c for c in tmp if c not in df_src.columns]
    if missing:
        add = pd.DataFrame({c: pd.Series(pd.NA, index=df_src.index) for c in missing})
        df_src = pd.concat([df_src, add], axis=1)
    return df_src.loc[:, list(tmp)]

# ----------------- Temporadas -----------------
def current_season_code(today=None):
    from datetime import datetime
    if today is None:
        today = datetime.now()
    y = today.year % 100
    start = y if today.month >= 7 else (y - 1) % 100
    end = (start + 1) % 100
    return f"{start:02d}{end:02d}"

def season_codes(first_start=5, last_code=None):
    if last_code is None:
        last_code = current_season_code()
    codes, y = [], first_start
    while True:
        code = f"{y:02d}{(y+1)%100:02d}"
        codes.append(code)
        if code == last_code:
            break
        y += 1
    return codes

# ----------------- Descarga FD -----------------
def fetch_fd_csv(season: str, div: str = "SP1") -> pd.DataFrame:
    base = "https://www.football-data.co.uk/mmz4281"
    url  = f"{base}/{season}/{div}.csv"
    r = requests.get(url, headers={"User-Agent":"Mozilla/5.0","Cache-Control":"no-cache"}, timeout=30)
    r.raise_for_status()
    df = pd.read_csv(io.BytesIO(r.content))
    return df

# ----------------- Normalización clave -----------------
def _deaccent(s: pd.Series) -> pd.Series:
    import unicodedata
    return s.astype(str).apply(lambda x: ''.join(c for c in unicodedata.normalize("NFKD", x) if not unicodedata.combining(c)))

def norm_str(s: pd.Series) -> pd.Series:
    return (
        _deaccent(s.fillna(""))
        .astype(str).str.strip().str.upper()
        .str.replace(r"\s+", " ", regex=True)
    )

ALIAS = {
    "ATL MADRID": "ATH MADRID",
    "ATLETICO MADRID": "ATH MADRID",
    "ATHLETIC BILBAO": "ATH BILBAO",
    "DEPORTIVO ALAVES": "ALAVES",
    "REAL SOCIEDAD": "SOCIEDAD",
    "REAL BETIS": "BETIS",
    "REAL VALLADOLID": "VALLADOLID",
    "CELTA VIGO": "CELTA",
    "ESPANOL": "ESPANYOL",
    "RCD ESPANYOL": "ESPANYOL",
    "RCD MALLORCA": "MALLORCA",
    "UD LAS PALMAS": "LAS PALMAS",
    "RAYO VALLECANO": "VALLECANO", "VALLECANO": "VALLECANO",
    "REAL OVIEDO": "OVIEDO", "SPORTING GIJON": "GIJON", "REAL SPORTING": "GIJON",
    "DEPORTIVO LA CORUNA": "DEPORTIVO", "REAL ZARAGOZA": "ZARAGOZA",
}

def canon_team_name(s: pd.Series) -> pd.Series:
    t = norm_str(s)
    t = t.str.replace(r"\b(CF|FC|SAD|CD|UD|RCDE|RCD|REAL CLUB DEPORTIVO|REAL CLUB)\b", "", regex=True)
    t = t.str.replace(r"\s+", " ", regex=True).str.strip()
    return t.map(lambda x: ALIAS.get(x, x))

_EMPTY_TOKENS = {"", "nan", "none", "<na>", "nul", "null"}
def _as_empty_na(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip().str.lower()
    return series.mask(s.isin(_EMPTY_TOKENS))

def _coalesce_team_cols(df: pd.DataFrame, base: str, fallback: str) -> pd.Series:
    a = df[base] if base in df.columns else pd.Series(pd.NA, index=df.index)
    b = df[fallback] if fallback in df.columns else pd.Series("", index=df.index)
    a = _as_empty_na(a); b = _as_empty_na(b)
    out = a.fillna(b)
    return out.where(~out.astype(str).str.strip().str.lower().isin(_EMPTY_TOKENS), "")

def _standardize_date_like_fd(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip()
    out = pd.Series([None]*len(s), dtype="object")
    iso = s.str.match(r"^\d{4}-\d{2}-\d{2}$")
    if iso.any():
        dt = pd.to_datetime(s[iso], format="%Y-%m-%d", errors="coerce")
        out.loc[iso] = dt.dt.strftime("%d/%m/%y")
    dmy4 = ~iso & s.str.match(r"^\d{1,2}/\d{1,2}/\d{4}$")
    if dmy4.any():
        dt = pd.to_datetime(s[dmy4], format="%d/%m/%Y", errors="coerce")
        out.loc[dmy4] = dt.dt.strftime("%d/%m/%y")
    dmy2 = ~iso & ~dmy4 & s.str.match(r"^\d{1,2}/\d{1,2}/\d{2}$")
    if dmy2.any():
        dt = pd.to_datetime(s[dmy2], format="%d/%m/%y", errors="coerce")
        out.loc[dmy2] = dt.dt.strftime("%d/%m/%y")
    rem = out.isna()
    if rem.any():
        dt = pd.to_datetime(s[rem], dayfirst=True, errors="coerce")
        out.loc[rem] = dt.dt.strftime("%d/%m/%y")
    return out.where(out.notna(), s)

def make_temp_key(df: pd.DataFrame) -> pd.Series:
    for c in ["Div","Date","HomeTeam","AwayTeam","HomeTeam_norm","AwayTeam_norm"]:
        if c not in df.columns:
            df[c] = pd.NA if c in ["HomeTeam","AwayTeam"] else ""
    div  = norm_str(df["Div"])
    date = norm_str(_standardize_date_like_fd(df["Date"]))
    home = canon_team_name(_coalesce_team_cols(df, "HomeTeam", "HomeTeam_norm"))
    away = canon_team_name(_coalesce_team_cols(df, "AwayTeam", "AwayTeam_norm"))
    return (div + "|" + date + "|" + home + "|" + away)

# ----------------- Esquema usado por MODELOS -----------------
MATCH_COLS_CORE = [
    "Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HTHG","HTAG","HTR",
    "HS","AS","HST","AST","HF","AF","HC","AC","HY","AY","HR","AR",
    "B365H","B365D","B365A"
]
MATCH_NUMERIC = [
    "FTHG","FTAG","HTHG","HTAG","HS","AS","HST","AST","HF","AF","HC","AC","HY","AY","HR","AR",
    "B365H","B365D","B365A"
]

def ensure_fd_master():
    if PARQUET_PATH.exists():
        return
    print("[BOOTSTRAP] Construyendo esquema inicial del maestro…")
    cols_union = set()
    for s in season_codes(first_start=5):
        try:
            df_s = fetch_fd_csv(s, "SP1")
            cols_union |= set(df_s.columns)
        except Exception as e:
            print("WARN al leer", s, "→", e)
    cols_union |= {"Div","Date","HomeTeam","AwayTeam"}
    cols_union |= set(FROZEN)
    cols_union |= set(MATCH_COLS_CORE)
    master_empty = pd.DataFrame(columns=sorted(cols_union))
    master_empty.to_parquet(PARQUET_PATH, index=False)
    print(f"[BOOTSTRAP] Maestro vacío creado en {PARQUET_PATH} con {len(master_empty.columns)} columnas")

# ----------------- MANUAL CUOTAS -----------------
MANUAL_DIR = Path("manual")
MANUAL_PATTERN = "b365_filled_*.csv"

def load_all_manual_b365(manual_dir=MANUAL_DIR, pattern=MANUAL_PATTERN) -> pd.DataFrame:
    paths = sorted(glob.glob(str(manual_dir / pattern)))
    print(f"[MANUAL] Archivos encontrados: {len(paths)}")
    for p in paths[:10]: print(" -", p)
    if not paths:
        return pd.DataFrame(columns=["_TMP_KEY_"] + FROZEN)

    dfs = []
    for p in paths:
        dfm = pd.read_csv(p)
        lower = {c.lower(): c for c in dfm.columns}
        def pick(*cands):
            for x in cands:
                if x in lower: return lower[x]
            return None
        ren = {}
        if (c:=pick("div")): ren[c]="Div"
        if (c:=pick("date")): ren[c]="Date"
        if (c:=pick("hometeam")): ren[c]="HomeTeam"
        if (c:=pick("awayteam")): ren[c]="AwayTeam"
        if (c:=pick("hometeam_norm","home_team_norm","home_norm")): ren[c]="HomeTeam_norm"
        if (c:=pick("awayteam_norm","away_team_norm","away_norm")): ren[c]="AwayTeam_norm"
        if (c:=pick("b365h")): ren[c]="B365H"
        if (c:=pick("b365d")): ren[c]="B365D"
        if (c:=pick("b365a")): ren[c]="B365A"
        dfm = dfm.rename(columns=ren)
        dfm = _dedup_columns(dfm, f"MANUAL CUOTAS:{Path(p).name}")
        if "Div" not in dfm: dfm["Div"] = "SP1"
        for c in FROZEN:
            if c in dfm.columns:
                dfm[c] = pd.to_numeric(dfm[c], errors="coerce")
        dfm["_TMP_KEY_"] = make_temp_key(dfm)
        dfs.append(dfm[["_TMP_KEY_"] + FROZEN])

    manual_all = pd.concat(dfs, ignore_index=True)
    manual_all = manual_all.drop_duplicates(subset=["_TMP_KEY_"], keep="last")
    return manual_all

# ----------------- MANUAL MATCHES (stats + opcional B365*) -----------------
MANUAL_MATCHES_PATTERN = "fd_matches_*.csv"

def _map_result_like_fd(s: pd.Series) -> pd.Series:
    m = s.astype(str).str.strip().str.upper().map({
        "H":"H","HOME":"H","1":"H",
        "D":"D","DRAW":"D","X":"D",
        "A":"A","AWAY":"A","2":"A",
    })
    return m.where(m.isin({"H","D","A"}), pd.NA)

def load_all_manual_matches(manual_dir=MANUAL_DIR, pattern=MANUAL_MATCHES_PATTERN) -> pd.DataFrame:
    paths = sorted(glob.glob(str(manual_dir / pattern)))
    print(f"[MANUAL MATCHES] Archivos encontrados: {len(paths)}")
    for p in paths[:10]: print(" -", p)
    if not paths:
        return pd.DataFrame(columns=["_TMP_KEY_","Div"] + MATCH_COLS_CORE)

    dfs = []
    for p in paths:
        dfm = pd.read_csv(p)
        lower = {c.lower(): c for c in dfm.columns}
        def pick(*cands):
            for x in cands:
                if x in lower: return lower[x]
            return None
        ren = {}
        if (c:=pick("div")): ren[c]="Div"
        if (c:=pick("date")): ren[c]="Date"
        if (c:=pick("hometeam")): ren[c]="HomeTeam"
        if (c:=pick("awayteam")): ren[c]="AwayTeam"
        if (c:=pick("hometeam_norm","home_team_norm","home_norm")): ren[c]="HomeTeam_norm"
        if (c:=pick("awayteam_norm","away_team_norm","away_norm")): ren[c]="AwayTeam_norm"
        for c_std in [c for c in MATCH_COLS_CORE if c not in {"Date","HomeTeam","AwayTeam"}]:
            if (c:=pick(c_std.lower())): ren[c]=c_std

        dfm = dfm.rename(columns=ren)
        dfm = _dedup_columns(dfm, f"MANUAL MATCHES:{Path(p).name}")

        if "Div" not in dfm: dfm["Div"] = "SP1"
        for c in ["HomeTeam","AwayTeam","HomeTeam_norm","AwayTeam_norm"]:
            if c not in dfm: dfm[c] = pd.NA
        ht = dfm["HomeTeam"].astype(str).str.strip()
        at = dfm["AwayTeam"].astype(str).str.strip()
        dfm.loc[ht.eq("") | ht.str.lower().isin(_EMPTY_TOKENS), "HomeTeam"] = dfm["HomeTeam_norm"]
        dfm.loc[at.eq("") | at.str.lower().isin(_EMPTY_TOKENS), "AwayTeam"] = dfm["AwayTeam_norm"]

        if "Date" not in dfm: dfm["Date"] = pd.NA
        dfm["Date"] = _standardize_date_like_fd(dfm["Date"].astype(str))
        if "FTR" in dfm: dfm["FTR"] = _map_result_like_fd(dfm["FTR"])
        if "HTR" in dfm: dfm["HTR"] = _map_result_like_fd(dfm["HTR"])

        for c in MATCH_NUMERIC:
            if c in dfm.columns:
                dfm[c] = pd.to_numeric(dfm[c], errors="coerce")

        dfm["_TMP_KEY_"] = make_temp_key(dfm)

        keep = ["_TMP_KEY_","Div","Date","HomeTeam","AwayTeam"] + \
              [c for c in MATCH_COLS_CORE if c in dfm.columns and c not in {"Date","HomeTeam","AwayTeam"}]
        # por si acaso, deduplicamos la lista manteniendo orden
        seen = set(); keep = [c for c in keep if not (c in seen or seen.add(c))]

        dfs.append(dfm[keep])

    manual_all = pd.concat(dfs, ignore_index=True)
    manual_all = manual_all.drop_duplicates(subset=["_TMP_KEY_"], keep="last")
    return manual_all

# ========================= EJECUCIÓN =========================
ensure_fd_master()

master = pd.read_parquet(PARQUET_PATH)
cols_master = list(master.columns)
master["_TMP_KEY_"] = make_temp_key(master)
pre_rows = len(master)
pre_keys = set(master["_TMP_KEY_"])

seasons = season_codes(first_start=5)

# 1) LIVE desde históricos FD
live_list = []
for season, div in product(seasons, DIVISIONS):
    df_season = fetch_fd_csv(season, div)
    for c in ["Div","Date","HomeTeam","AwayTeam"]:
        if c in df_season.columns:
            df_season[c] = df_season[c].fillna("").astype(str)
    df_season["_TMP_KEY_"] = make_temp_key(df_season)
    df_season = df_season.drop_duplicates(subset=["_TMP_KEY_"], keep="last")
    live_list.append(df_season)

if not live_list:
    print("No se descargó nada nuevo.")
    master = master.drop(columns=["_TMP_KEY_"], errors="ignore")
    master.to_parquet(PARQUET_PATH, index=False)
else:
    live = pd.concat(live_list, ignore_index=True)
    live = _dedup_columns(live, "LIVE")

    # Asegura columnas del master
    extra_cols = [c for c in cols_master if c not in live.columns]
    if extra_cols:
        live = pd.concat([live, pd.DataFrame(columns=extra_cols)], axis=1)
    if "_TMP_KEY_" not in live.columns:
        live["_TMP_KEY_"] = make_temp_key(live)

    # === INYECCIÓN DE PARTIDOS MANUALES (stats + opcional B365*) ===
    man_matches = load_all_manual_matches()
    if not man_matches.empty:
        live_keys = set(live["_TMP_KEY_"].astype(str))
        man_keys  = set(man_matches["_TMP_KEY_"].astype(str))

        # 1) Añadir partidos que no existan aún en históricos
        new_keys = man_keys - live_keys
        if new_keys:
            to_add = man_matches.loc[man_matches["_TMP_KEY_"].isin(new_keys)].copy()
            to_add = align_columns_like(to_add, live.columns)   # <- sin reindex()
            live = pd.concat([live, to_add], ignore_index=True)
            print(f"[MANUAL MATCHES] Añadidos {len(to_add)} partidos nuevos (no estaban en históricos).")

        # 2) Override de columnas disponibles en man_matches (incluye B365* si vinieran)
        # Selección SIN duplicar nombres
        cols_for_merge = ["_TMP_KEY_","Div","Date","HomeTeam","AwayTeam"] + \
                        [c for c in MATCH_COLS_CORE if c in man_matches.columns and c not in {"Date","HomeTeam","AwayTeam"}]
        # dedup preservando orden
        _seen = set(); cols_for_merge = [c for c in cols_for_merge if not (c in _seen or _seen.add(c))]

        right = man_matches[cols_for_merge].copy()
        right = _dedup_columns(right, "MMAN.SELECT")

        live = live.merge(right, on="_TMP_KEY_", how="left", suffixes=("", "_MMAN"))
        live = _dedup_columns(live, "LIVE.POST.MERGE")

        applied = {}
        for col in MATCH_COLS_CORE:
            mcol = f"{col}_MMAN"
            if mcol not in live.columns:
                continue
            # Si por cualquier motivo mcol aparece duplicado y pandas devuelve DF, nos quedamos con la última
            mser = live[mcol]
            if isinstance(mser, pd.DataFrame):
                mser = mser.iloc[:, -1]
            mask = mser.notna()
            if str(mser.dtype) == "object":
                mask &= mser.astype(str).str.strip().ne("")
            if mask.any():
                live.loc[mask, col] = mser[mask]
                applied[col] = int(mask.sum())
            # limpia columna auxiliar
            live.drop(columns=[mcol], inplace=True, errors="ignore")
            # tipado numérico si procede
            if col in MATCH_NUMERIC and col in live.columns:
                live[col] = pd.to_numeric(live[col], errors="coerce")

        print("[MANUAL MATCHES] Overrides aplicados por columna:", applied)
    else:
        print("[MANUAL MATCHES] No se encontraron CSVs de partidos manuales.")

    # 2) Overrides MANUAL de cuotas (última palabra para B365*)
    manual = load_all_manual_b365()
    manual_keys = set(manual["_TMP_KEY_"]) if not manual.empty else set()

    if not manual.empty:
        live_keys = set(live["_TMP_KEY_"].astype(str))
        man_keys  = set(manual["_TMP_KEY_"].astype(str))
        missing_in_live = sorted(man_keys - live_keys)
        if missing_in_live:
            print(f"[DIAGNÓSTICO] {len(missing_in_live)} claves del MANUAL (cuotas) no existen en live (no se aplicarán):")
            for k in missing_in_live[:10]:
                print("  -", k)
            if len(missing_in_live) > 10:
                print(f"  ... y {len(missing_in_live)-10} más")

    if manual_keys:
        before_live = live[FROZEN].copy() if set(FROZEN).issubset(live.columns) else pd.DataFrame()
        live = live.merge(manual, on="_TMP_KEY_", how="left", suffixes=("", "_MAN"))

        overrides = {}
        for col in FROZEN:
            man_col = f"{col}_MAN"
            if man_col in live.columns:
                mask = live[man_col].notna()
                overrides[col] = int(mask.sum())
                live.loc[mask, col] = live.loc[mask, man_col]
                live.drop(columns=[man_col], inplace=True, errors="ignore")
            live[col] = pd.to_numeric(live[col], errors="coerce")

        live["_FROM_MANUAL"] = live["_TMP_KEY_"].isin(manual_keys)

        changed = {}
        if not before_live.empty:
            for col in FROZEN:
                changed[col] = int(((before_live[col] != live[col]) & live[col].notna()).fillna(False).sum())
        else:
            changed = {col: int(live["_FROM_MANUAL"].sum()) for col in FROZEN}

        print("[MANUAL B365] Filas con valor manual aplicado:", int(live["_FROM_MANUAL"].sum()))
        print("[MANUAL B365] Aplicados por columna:", overrides)
        print("[MANUAL B365] Cambios efectivos vs FD:", changed)
    else:
        live["_FROM_MANUAL"] = False
        print("[MANUAL B365] No se encontraron ficheros manuales.")

    # 3) MERGE con master priorizando cuotas manuales
    if not master.empty:
        prev = master[["_TMP_KEY_"] + [c for c in FROZEN if c in cols_master]].copy()
        merged = live.merge(prev, on="_TMP_KEY_", how="left", suffixes=("", "_OLD"))

        mask_man = merged["_FROM_MANUAL"].fillna(False)
        for col in FROZEN:
            old = f"{col}_OLD"
            if old in merged.columns:
                use_old = (~mask_man) & merged[old].notna()
                merged.loc[use_old, col] = merged.loc[use_old, old]
        merged = merged.drop(columns=[c for c in merged.columns if c.endswith("_OLD")], errors="ignore")
    else:
        merged = live.copy()

    # 4) Consolidación + guardado
    to_keep_old = master[~master["_TMP_KEY_"].isin(set(merged["_TMP_KEY_"]))].copy()
    updated = merged.copy()

    combo = pd.concat([to_keep_old, updated], ignore_index=True)
    combo = combo.drop_duplicates(subset=["_TMP_KEY_"], keep="last")
    missing_in_combo = [c for c in cols_master if c not in combo.columns]
    if missing_in_combo:
        combo = pd.concat([combo, pd.DataFrame(columns=missing_in_combo)], axis=1)
    combo = combo[cols_master + (["_TMP_KEY_"] if "_TMP_KEY_" in combo.columns else [])]
    combo = combo.drop(columns=["_TMP_KEY_","_FROM_MANUAL"], errors="ignore")
    combo.to_parquet(PARQUET_PATH, index=False)

    post_rows = len(combo)
    post_keys = set(make_temp_key(combo))
    added   = len(post_keys - pre_keys)
    touched = len(post_keys & pre_keys)

    print(f"Estructura mantenida => {len(cols_master)} columnas.")
    print(f"Partidos antes: {pre_rows:,} | ahora: {post_rows:,} | nuevos: {added:,} | coincidentes: {touched:,}")

    # 5) VALIDACIÓN final de cuotas vs manual
    if manual_keys:
        final = combo.copy()
        final["_TMP_KEY_"] = make_temp_key(final)
        chk = final[["_TMP_KEY_"] + FROZEN].merge(manual, on="_TMP_KEY_", how="inner", suffixes=("_FINAL","_MAN"))
        mismatches = {}
        for col in FROZEN:
            c_final, c_man = f"{col}_FINAL", f"{col}_MAN"
            diff = (pd.to_numeric(chk[c_final], errors="coerce") - pd.to_numeric(chk[c_man], errors="coerce")).abs()
            mismatches[col] = int((diff > 1e-9).sum())
        print("[VALIDACIÓN] Discrepancias final vs manual por columna:", mismatches)
        if sum(mismatches.values()) > 0:
            bad_rows = []
            for col in FROZEN:
                c_final, c_man = f"{col}_FINAL", f"{col}_MAN"
                diff_mask = (pd.to_numeric(chk[c_final], errors="coerce") - pd.to_numeric(chk[c_man], errors="coerce")).abs() > 1e-9
                bad_rows.append(chk.loc[diff_mask, ["_TMP_KEY_", c_final, c_man]].assign(COL=col))
            bad = pd.concat(bad_rows, ignore_index=True) if bad_rows else pd.DataFrame()
            print("[VALIDACIÓN] Ejemplos de discrepancias (máx 5):")
            if not bad.empty:
                print(bad.head(5).to_string(index=False))
            else:
                print("—")

[BOOTSTRAP] Construyendo esquema inicial del maestro…
[BOOTSTRAP] Maestro vacío creado en /content/data/02_processed/football-data.co.uk_2005_2025.parquet con 191 columnas
[MANUAL MATCHES] Archivos encontrados: 0
[MANUAL MATCHES] No se encontraron CSVs de partidos manuales.
[MANUAL] Archivos encontrados: 12
 - manual/b365_filled_2025-08-15.csv
 - manual/b365_filled_2025-09-15.csv
 - manual/b365_filled_2025-09-28.csv
 - manual/b365_filled_2025-10-01.csv
 - manual/b365_filled_2025-10-16.csv
 - manual/b365_filled_2025-10-21.csv
 - manual/b365_filled_2025-10-28.csv
 - manual/b365_filled_2025-11-04.csv
 - manual/b365_filled_2025-11-10.csv
 - manual/b365_filled_2025-11-25.csv
[MANUAL B365] Filas con valor manual aplicado: 106
[MANUAL B365] Aplicados por columna: {'B365H': 106, 'B365D': 106, 'B365A': 106}
[MANUAL B365] Cambios efectivos vs FD: {'B365H': 71, 'B365D': 59, 'B365A': 72}


[12/12/25 17:39:48] WARNING  /tmp/ipython-input-3438005975.py:445: FutureWarning: The behavior of   ]8;id=315189;file:///usr/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=601392;file:///usr/lib/python3.12/warnings.py#112\112]8;;\
                             DataFrame concatenation with empty or all-NA entries is deprecated. In                
                             a future version, this will no longer exclude empty or all-NA columns                 
                             when determining the result dtypes. To retain the old behavior,                       
                             exclude the relevant entries before the concat operation.                             
                               combo = pd.concat(, ignore_index=True)                                              
                                                                                                                   

Estructura mantenida => 191 columnas.
Partidos antes: 0 | ahora: 7,752 | nuevos: 7,752 | coincidentes: 0
[VALIDACIÓN] Discrepancias final vs manual por columna: {'B365H': 0, 'B365D': 0, 'B365A': 0}


In [8]:
fd = pd.read_parquet(PARQUET_PATH)

fd

,1XBA,1XBCA,1XBCD,1XBCH,1XBD,1XBH,AC,AF,AHCh,AHh,...,VCCD,VCCH,VCD,VCH,WHA,WHCA,WHCD,WHCH,WHD,WHH
0,NaN,NaN,NaN,NaN,NaN,NaN,7,19,NaN,NaN,...,NaN,NaN,3.75,6.50,1.50,NaN,NaN,NaN,3.4,6.00
1,NaN,NaN,NaN,NaN,NaN,NaN,4,19,NaN,NaN,...,NaN,NaN,NaN,NaN,3.75,NaN,NaN,NaN,3.0,1.90
2,NaN,NaN,NaN,NaN,NaN,NaN,5,14,NaN,NaN,...,NaN,NaN,3.25,1.85,3.50,NaN,NaN,NaN,3.1,1.95
3,NaN,NaN,NaN,NaN,NaN,NaN,4,22,NaN,NaN,...,NaN,NaN,3.40,1.65,4.50,NaN,NaN,NaN,3.2,1.70
4,NaN,NaN,NaN,NaN,NaN,NaN,8,25,NaN,NaN,...,NaN,NaN,3.60,6.50,1.44,NaN,NaN,NaN,3.6,6.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7747,NaN,NaN,NaN,NaN,NaN,NaN,2,13,-0.25,-0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7748,NaN,NaN,NaN,NaN,NaN,NaN,2,18,-0.50,-0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7749,NaN,NaN,NaN,NaN,NaN,NaN,2,22,-0.25,-0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7750,NaN,NaN,NaN,NaN,NaN,NaN,1,12,-1.75,-1.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2. Understat (xG)

In [10]:
# # ╔════════════════════════════════════╗
# # ║  UNDERSTAT  •  La Liga  •  xG      ║
# # ╚════════════════════════════════════╝

# PROC.mkdir(parents=True, exist_ok=True)
# SAVE_PATH = PROC / "understat_2014_2025.parquet"

# # CAMBIAR PARA AÑADIR DATOS FUTUROS
# UNDER_SEASONS = [1415, 1516, 1617, 1718, 1819, 1920, 2021, 2122, 2223, 2324, 2425, 2526]

# us = sd.Understat(leagues="ESP-La Liga", seasons=UNDER_SEASONS)

# team_stats = us.read_team_match_stats()
# print("Rows team_stats:", len(team_stats))

# xg_df = (
#     team_stats[[
#         "game_id", "date",
#         "home_team", "away_team",
#         "home_xg",  "away_xg"
#     ]]
#     .rename(columns={
#         "game_id"   : "match_id",
#         "date"      : "Date",
#         "home_team" : "HomeTeam",
#         "away_team" : "AwayTeam",
#         "home_xg"   : "h_xg",
#         "away_xg"   : "a_xg"
#     })
# )
# xg_df["Date"] = pd.to_datetime(xg_df["Date"]).dt.date

# xg_df.to_parquet(SAVE_PATH, index=False)
# print("Guardado:", SAVE_PATH)
# print("Partidos con xG:", len(xg_df))
# xg_df.tail(10)

In [11]:
FD_PATH = PROC / "football-data.co.uk_2005_2025.parquet"
XG_PATH = PROC / "understat_2014_2025.parquet"

fd = pd.read_parquet(FD_PATH)
xg = pd.read_parquet(XG_PATH)

print("FD partidos:", len(fd))
print("xG partidos:", len(xg))

FD partidos: 7752
xG partidos: 3192


Las tablas que queremos juntar tienen diferente formato de fechas por lo que primero que hacemos es unificarlos. Prevenimos el problema de que en muchos casos el año puede venir en dos y en cuatro digitos.

In [12]:
fd["Date"] = pd.to_datetime(
    fd["Date"].astype(str).str.strip(),
    format="%d/%m/%y", errors="coerce"
).fillna(
    pd.to_datetime(fd["Date"], format="%d/%m/%Y", errors="coerce")
).dt.date

Además vemos que cada tabla usa nombres diferentes para los equipos.

In [13]:
def norm(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode("utf-8")
    s = re.sub(r"[. ]+", " ", s).lower().strip()
    return s

for col in ["HomeTeam", "AwayTeam"]:
    fd[col+"_norm"] = fd[col].apply(norm)
    xg[col+"_norm"] = xg[col].apply(norm)

alias = {
    # Understat → football-data
    "real oviedo"        : "oviedo",
    "almeria"            : "almeria",
    "granada"            : "granada",
    "malaga"             : "malaga",
    "sevilla"            : "sevilla",
    "barcelona"          : "barcelona",
    "celta vigo"         : "celta",
    "eibar"              : "eibar",
    "levante"            : "levante",
    "rayo vallecano"     : "vallecano",
    "real madrid"        : "real madrid",
    "getafe"             : "getafe",
    "valencia"           : "valencia",
    "athletic club"      : "ath bilbao",
    "atletico madrid"    : "ath madrid",
    "cordoba"            : "cordoba",
    "espanyol"           : "espanol",
    "deportivo la coruna": "la coruna",
    "elche"              : "elche",
    "real sociedad"      : "sociedad",
    "villarreal"         : "villarreal",
    "sporting gijon"     : "sp gijon",
    "real betis"         : "betis",
    "las palmas"         : "las palmas",
    "osasuna"            : "osasuna",
    "alaves"             : "alaves",
    "leganes"            : "leganes",
    "girona"             : "girona",
    "real valladolid"    : "valladolid",
    "sd huesca"          : "huesca",
    "mallorca"           : "mallorca",
    "cadiz"              : "cadiz",
    "zaragoza"           : "zaragoza",
    "recreativo"         : "recreativo",
    "gimnastic"          : "gimnastic",
    "murcia"             : "murcia",
    "numancia"           : "numancia",
    "xerez"              : "xerez",
    "tenerife"           : "tenerife",
    "santander"          : "santander",
    "hercules"           : "hercules"
}
xg["HomeTeam_norm"] = xg["HomeTeam_norm"].replace(alias)
xg["AwayTeam_norm"] = xg["AwayTeam_norm"].replace(alias)

Ahora las variables "HomeTeam_norm" y "AwayTeam_norm" de ambos datasets tienen los mismos nombres de equipos.

In [14]:
key = ["Date", "HomeTeam_norm", "AwayTeam_norm"]

merged = (
    fd.merge(
        xg[key + ["h_xg", "a_xg"]],
        on=key,
        how="left",
        validate="many_to_one"
    )
)

print("Partidos totales:", len(merged))
print("Partidos con xG:", merged["h_xg"].notna().sum())

Partidos totales: 7752
Partidos con xG: 3192


Nos fijamos que faltan partidos por emparejar entre los dos datasets.

In [15]:
missing = (
    xg.merge(
        merged[key],
        on=key, how="left", indicator=True
    )
    .query("_merge == 'left_only'")
    .drop(columns="_merge")
)

print("Understat sin pareja exacta:", len(missing))
display(missing.head(15))

Understat sin pareja exacta: 0


,match_id,Date,HomeTeam,AwayTeam,h_xg,a_xg,HomeTeam_norm,AwayTeam_norm


Analizando por que fallaban estos partidos, las fechas no son correctas por uno o dos días de diferencia. Se hizo lo siguiente:

In [16]:
xg_cols = ["h_xg","a_xg"]

def fill_by_shift(df_base: pd.DataFrame, df_xg: pd.DataFrame, shift_days: int):
    tmp = df_xg.copy()
    tmp["Date"] = tmp["Date"] + timedelta(days=shift_days)

    tmp = tmp[key + xg_cols]
    tmp = tmp.rename(columns={c: f"{c}_sh{shift_days:+d}" for c in xg_cols})

    joined = df_base.merge(tmp, on=key, how="left")

    mask = joined["h_xg"].isna() & joined[f"h_xg_sh{shift_days:+d}"].notna()
    for c in xg_cols:
        joined.loc[mask, c] = joined.loc[mask, f"{c}_sh{shift_days:+d}"]

    joined = joined.drop(columns=[f"{c}_sh{shift_days:+d}" for c in xg_cols])
    return joined

for d in [1, -1, 2, -2]:
    merged = fill_by_shift(merged, xg, d)

print("Emparejados tras fechas flexibles:", merged["h_xg"].notna().sum())

Emparejados tras fechas flexibles: 3192


In [17]:
PROC.mkdir(parents=True, exist_ok=True)

output_path = PROC / "fd_xg_2005_2025.parquet"
merged.to_parquet(output_path, index=False)
print(f"Guardado: {output_path} · filas={len(merged):,}")

Guardado: /content/data/02_processed/fd_xg_2005_2025.parquet · filas=7,752


# 3. Rating ClubElo

In [ ]:
# ╔════════════════════════════════════════╗
# ║  CLUBELO  •  La Liga 2005-25  •  ELO   ║
# ╚════════════════════════════════════════╝

# PROC.mkdir(parents=True, exist_ok=True)
# SAVE_PATH = PROC / "clubelo_2005_2025.parquet"

# # Lista de nombres exactos tipo ClubElo
# CLUBS = [
#     "Real Madrid","Barcelona","Atletico","Bilbao","Sevilla",
#     "Valencia","Villarreal","Sociedad","Betis","Osasuna","Espanyol",
#     "Getafe","Celta","Mallorca","Las Palmas","Cadiz","Almeria","Granada",
#     "Alaves","Levante","Rayo Vallecano","Eibar","Girona","Leganes","Huesca",
#     "Valladolid","Elche","Cordoba","Gijon","Depor", "Malaga", "Oviedo",
#     "Zaragoza","Xerez","Tenerife","Recreativo","Numancia","Murcia",
#     "Tarragona","Santander","Hercules"
# ]

# def norm_elo(s):
#     return norm(s.replace(" ",""))

# ce = sd.ClubElo()
# frames = []

# for club in CLUBS:
#     try:
#         hist = ce.read_team_history(club)
#         if hist.empty:
#             print("Sin datos", club); continue

#         hist = hist.reset_index().rename(columns={hist.index.name or "index":"Date"})
#         rating_col = "elo" if "elo" in hist.columns else "Elo"
#         df = hist[["Date", rating_col]].rename(columns={rating_col:"Elo"})
#         df["Team"]   = club
#         df["team_norm"] = norm(club)
#         frames.append(df)
#         print("✓", club, "filas:", len(df))
#     except Exception as e:
#         print("X", club, "→", e)
#         time.sleep(3)

# elo_es = pd.concat(frames, ignore_index=True)
# elo_es["Date"] = pd.to_datetime(elo_es["Date"]).dt.date

# elo_es.to_parquet(SAVE_PATH, index=False)
# print(f"Filas Elo España: {len(elo_es)}")
# print(f"Guardado en {SAVE_PATH}")

In [ ]:
# ============================================================
# CLUB ELO • Update incremental "hasta hoy" del parquet existente
# ============================================================

from pathlib import Path
import time
import pandas as pd
import soccerdata as sd  # asumiendo que ya lo tenías importado así

# ---------- RUTAS ----------
PROC = Path("./data/02_processed")
PROC.mkdir(parents=True, exist_ok=True)
SAVE_PATH = PROC / "clubelo_2005_2025.parquet"
TMP_PATH  = PROC / (SAVE_PATH.name + ".tmp")

# ---------- CLUBS (histórico completo, por si los necesitas alguna vez) ----------
CLUBS = [
    "Real Madrid","Barcelona","Atletico","Bilbao","Sevilla","Valencia","Villarreal","Sociedad",
    "Betis","Osasuna","Espanyol","Getafe","Celta","Mallorca","Las Palmas","Cadiz","Almeria",
    "Granada","Alaves","Levante","Rayo Vallecano","Eibar","Girona","Leganes","Huesca",
    "Valladolid","Elche","Cordoba","Gijon","Depor","Malaga","Oviedo","Zaragoza","Xerez",
    "Tenerife","Recreativo","Numancia","Murcia","Tarragona","Santander","Hercules"
]

# ---------- CLUBS PRIMERA ACTUAL (solo estos se actualizan en cada run) ----------
CLUBS_PRIMERA = [
    "Real Madrid","Barcelona","Atletico","Bilbao","Sevilla","Valencia","Villarreal","Sociedad",
    "Betis","Osasuna","Espanyol","Getafe","Celta","Mallorca","Alaves","Levante",
    "Rayo Vallecano","Girona","Elche","Oviedo"
]

# ---------- helpers ----------
def norm(s: str) -> str:
    """Normaliza a 'team_norm' simple (sin acentos, minúsculas, _)."""
    import unicodedata, re
    s = "" if s is None else str(s)
    t = unicodedata.normalize("NFKD", s)
    t = "".join(c for c in t if not unicodedata.combining(c))
    return re.sub(r"[^A-Za-z0-9]+", " ", t).strip().lower().replace(" ", "_")

def _load_existing(path: Path) -> pd.DataFrame:
    if not path.exists():
        return pd.DataFrame(columns=["Date","Elo","Team","team_norm"])
    df = pd.read_parquet(path)
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"]).dt.date
    return df

def _fetch_history_one(club: str, ce: sd.ClubElo) -> pd.DataFrame:
    """
    Descarga TODO el histórico del club (ClubElo no filtra por fecha)
    y devuelve esquema estándar.
    """
    hist = ce.read_team_history(club)
    if hist is None or hist.empty:
        return pd.DataFrame(columns=["Date","Elo","Team","team_norm"])

    hist = hist.reset_index().rename(columns={hist.index.name or "index": "Date"})
    rating_col = "elo" if "elo" in hist.columns else ("Elo" if "Elo" in hist.columns else None)
    if rating_col is None:
        return pd.DataFrame(columns=["Date","Elo","Team","team_norm"])

    out = hist[["Date", rating_col]].rename(columns={rating_col: "Elo"}).copy()
    out["Team"] = club
    out["team_norm"] = norm(club)
    out["Date"] = pd.to_datetime(out["Date"]).dt.date
    return out

def _append_new_rows(existing: pd.DataFrame, incoming: pd.DataFrame) -> pd.DataFrame:
    """Concat + dedupe por (Team, Date), conservando la última."""
    if existing.empty:
        base = incoming.copy()
    else:
        base = pd.concat([existing, incoming], ignore_index=True)
    base = (
        base.sort_values(["Team", "Date"])
            .drop_duplicates(subset=["Team","Date"], keep="last")
    )
    return base.sort_values(["Team","Date"]).reset_index(drop=True)

# ---------- UPDATE INCREMENTAL HASTA HOY ----------
def update_elo_until_today(clubs=None, sleep_ok: float = 0.4, verbose: bool = True) -> pd.DataFrame:
    """
    Actualiza Elo hasta hoy SOLO para los clubes indicados en `clubs`.
    Por defecto usa la lista de equipos de Primera División actual (CLUBS_PRIMERA),
    pero conserva el histórico de todos los que haya en el parquet.
    """
    if clubs is None:
        clubs = CLUBS_PRIMERA  # <- por defecto, solo Primera actual

    existing = _load_existing(SAVE_PATH)
    last_by_team = (
        existing.groupby("Team")["Date"].max()
        if not existing.empty
        else pd.Series(dtype="object")
    )

    ce = sd.ClubElo()  # una sola instancia para todos los equipos
    updates = []

    for club in clubs:
        try:
            df_hist = _fetch_history_one(club, ce)
            if df_hist.empty:
                if verbose:
                    print(f"– Sin datos: {club}")
                continue

            last_date = last_by_team.get(club, None)
            if last_date is not None and pd.notna(last_date):
                inc = df_hist[df_hist["Date"] > last_date]
            else:
                inc = df_hist

            if not inc.empty:
                updates.append(inc)
                if verbose:
                    print(
                        f"✓ {club}: +{len(inc)} filas nuevas "
                        f"(de {inc['Date'].min()} a {inc['Date'].max()})"
                    )
            else:
                if verbose:
                    print(f"= {club}: sin novedades")

            if sleep_ok:
                time.sleep(sleep_ok)
        except Exception as e:
            print(f"X {club}: {e}")
            time.sleep(1)

    if not updates:
        if verbose:
            print("No hay filas nuevas para ningún club de la lista. Parquet ya está al día.")
        return existing

    incoming = pd.concat(updates, ignore_index=True)
    updated  = _append_new_rows(existing, incoming)

    # escritura atómica
    updated.to_parquet(TMP_PATH, index=False)
    TMP_PATH.replace(SAVE_PATH)

    if verbose:
        print(f"Actualizado: {len(updated)} filas totales → {SAVE_PATH}")

    return updated

# --- Ejecutar ---
if __name__ == "__main__":
    # explícitamente solo equipos de Primera actual
    _ = update_elo_until_today(clubs=CLUBS_PRIMERA)

[12/04/25 17:41:59] INFO     Saving cached data to /root/soccerdata/data/ClubElo                     ]8;id=83023;file:///usr/local/lib/python3.12/dist-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=75832;file:///usr/local/lib/python3.12/dist-packages/soccerdata/_common.py#263\263]8;;\

✓ Real Madrid: +14 filas nuevas (de 2025-11-02 a 2025-12-15)
✓ Barcelona: +17 filas nuevas (de 2025-10-30 a 2025-12-15)
✓ Atletico: +17 filas nuevas (de 2025-11-02 a 2025-12-15)
✓ Bilbao: +16 filas nuevas (de 2025-11-02 a 2025-12-15)
✓ Sevilla: +16 filas nuevas (de 2025-11-01 a 2025-12-15)
✓ Valencia: +20 filas nuevas (de 2025-10-30 a 2025-12-15)
✓ Villarreal: +15 filas nuevas (de 2025-11-02 a 2025-12-15)
✓ Sociedad: +18 filas nuevas (de 2025-11-02 a 2025-12-15)
✓ Betis: +19 filas nuevas (de 2025-10-30 a 2025-12-15)
✓ Osasuna: +16 filas nuevas (de 2025-11-02 a 2025-12-15)
✓ Espanyol: +19 filas nuevas (de 2025-11-02 a 2025-12-15)
✓ Getafe: +19 filas nuevas (de 2025-11-01 a 2025-12-15)
✓ Celta: +16 filas nuevas (de 2025-10-30 a 2025-12-15)
✓ Mallorca: +17 filas nuevas (de 2025-11-03 a 2025-12-15)
✓ Alaves: +16 filas nuevas (de 2025-11-01 a 2025-12-15)
✓ Levante: +17 filas nuevas (de 2025-10-30 a 2025-12-15)
✓ Rayo Vallecano: +17 filas nuevas (de 2025-11-02 a 2025-12-15)
✓ Girona: +16 fil

In [ ]:
PROC.mkdir(parents=True, exist_ok=True)

df = pd.read_parquet(PROC / "fd_xg_2005_2025.parquet")
df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()

elo_es = pd.read_parquet(PROC / "clubelo_2005_2025.parquet")
elo_es["Date"] = pd.to_datetime(elo_es["Date"]).dt.normalize()

clubelo_to_fd = {
    'real madrid': 'real madrid',
    'real oviedo'    : 'real oviedo',
    'barcelona': 'barcelona',
    'atletico': 'ath madrid',
    'bilbao': 'ath bilbao',
    'sevilla': 'sevilla',
    'valencia': 'valencia',
    'villarreal': 'villarreal',
    'sociedad': 'sociedad',
    'betis': 'betis',
    'osasuna': 'osasuna',
    'espanyol': 'espanol',
    'getafe': 'getafe',
    'celta': 'celta',
    'mallorca': 'mallorca',
    'las palmas': 'las palmas',
    'cadiz': 'cadiz',
    'almeria': 'almeria',
    'granada': 'granada',
    'alaves': 'alaves',
    'levante': 'levante',
    'rayo vallecano': 'vallecano',
    'eibar': 'eibar',
    'girona': 'girona',
    'leganes': 'leganes',
    'huesca': 'huesca',
    'valladolid': 'valladolid',
    'elche': 'elche',
    'cordoba': 'cordoba',
    'gijon': 'sp gijon',
    'depor': 'la coruna',
    'malaga': 'malaga',
    'zaragoza': 'zaragoza',
    'xerez': 'xerez',
    'tenerife': 'tenerife',
    'recreativo': 'recreativo',
    'numancia': 'numancia',
    'murcia': 'murcia',
    'tarragona': 'gimnastic',
    'santander': 'santander',
    'hercules': 'hercules'
}
elo_es['team_norm'] = elo_es['team_norm'].replace(clubelo_to_fd)

elo_home = elo_es.rename(columns={"team_norm":"HomeTeam_norm","Elo":"h_elo"}).sort_values("Date")
elo_away = elo_es.rename(columns={"team_norm":"AwayTeam_norm","Elo":"a_elo"}).sort_values("Date")

df = pd.merge_asof(
        df.sort_values("Date"),
        elo_home[["Date","HomeTeam_norm","h_elo"]],
        on="Date", by="HomeTeam_norm", direction="backward"
)

df = pd.merge_asof(
        df.sort_values("Date"),
        elo_away[["Date","AwayTeam_norm","a_elo"]],
        on="Date", by="AwayTeam_norm", direction="backward"
)

print("Cobertura h_elo:", df['h_elo'].notna().mean()*100, "%")
print("Cobertura a_elo:", df['a_elo'].notna().mean()*100, "%")

SAVE_PATH = PROC / "fd_xg_elo_2005_2025.parquet"
df.to_parquet(SAVE_PATH, index=False)
print(f"Guardado {SAVE_PATH}")

Cobertura h_elo: 100.0 %
Cobertura a_elo: 100.0 %
Guardado data/02_processed/fd_xg_elo_2005_2025.parquet


# 4. Transfermarkt (Plantilla y Mercado)

## **+ 6 HORAS (Mejor no ejecutar)**

In [ ]:
# # ╔════════════════════════════════════════════╗
# # ║  TRANSFERMARKT  •  La Liga 2005-25  •  €€  ║
# # ╚════════════════════════════════════════════╝

# HEADERS = {
#     "User-Agent": UserAgent().random,
#     "Accept-Language": "es-ES,es;q=0.9,en;q=0.8",
#     "Referer": "https://www.transfermarkt.com/"
# }
# BASE = "https://www.transfermarkt.com"
# DELAY = 6 + np.random.uniform(0, 5)

# def fetch_html(url, retries=4, base_delay=7, factor=2):
#     for i in range(retries):
#         try:
#             r = requests.get(url, headers=HEADERS, timeout=40)
#             if r.status_code == 200:
#                 return r.text
#             print(f"⟳  Retry {i+1}/{retries}  status {r.status_code}")
#         except Exception as e:
#             print(f"⟳  Retry {i+1}/{retries}  error: {e}")
#         delay = base_delay * (factor ** i) + np.random.uniform(0, 5)
#         print(f"Sleeping {delay:.1f}s before retry {i+1}")
#         time.sleep(delay)
#     return None

# def parse_euro_value(text):
#     """
#     Devuelve el valor en millones de euros (M€) desde el formato Transfermarkt (.es y .com).
#     """
#     text = text.strip().replace('\xa0', '').replace(' ', '').lower()
#     num_match = re.search(r"([\d.,]+)", text)
#     if not num_match:
#         return None

#     value_str = num_match.group(1)
#     if "." in value_str and "," in value_str:
#         value_str = value_str.replace(".", "").replace(",", ".")
#     elif "." in value_str:
#         value_str = value_str.replace(".", "")
#     elif "," in value_str:
#         value_str = value_str.replace(",", ".")
#     try:
#         value = float(value_str)
#     except:
#         return None

#     if "mill" in text or "million" in text or "mio" in text or re.search(r"\bm\b", text):
#         return value
#     elif value >= 10000:
#         return value / 1000
#     elif value >= 1000:
#         return value / 1000
#     else:
#         return value / 1_000_000

# def parse_squad_table(soup):
#     """
#     Extrae el número de jugadores (squad size) y el % de extranjeros (primera nacionalidad ≠ España)
#     """
#     table = soup.find("table", class_="items")
#     if not table:
#         return None, None

#     tbody = table.find("tbody")
#     if not tbody:
#         return None, None

#     rows = tbody.find_all("tr", recursive=False)
#     squad_size = 0
#     n_extranjeros = 0

#     for row in rows:
#         cells = row.find_all("td", recursive=False)
#         if len(cells) < 4:
#             continue

#         nat_cell = cells[3]
#         flags = nat_cell.find_all("img", class_="flaggenrahmen")
#         if flags:
#             primera_nacionalidad = flags[0].get("title", "").strip()
#             if primera_nacionalidad not in ["España", "Spain"]:
#                 n_extranjeros += 1
#         else:
#             n_extranjeros += 1

#         squad_size += 1

#     pct_extranjeros = round(100 * n_extranjeros / squad_size, 2) if squad_size > 0 else None
#     return squad_size, pct_extranjeros

# def parse_tm_row_summary(soup):
#     tfoot = soup.find("tfoot")
#     avg_age = total_value = avg_value = None
#     if tfoot:
#         row = tfoot.find("tr")
#         age_td = row.find("td", class_="zentriert")
#         if age_td:
#             try:
#                 avg_age = float(age_td.get_text(strip=True).replace(",", "."))
#             except:
#                 avg_age = None
#         rechts_tds = row.find_all("td", class_="rechts")
#         if len(rechts_tds) >= 3:
#             total_value = parse_euro_value(rechts_tds[1].get_text(strip=True))
#             avg_value = parse_euro_value(rechts_tds[2].get_text(strip=True))
#     return avg_age, total_value, avg_value

# def scrape_tm(team_slug, team_id, season):
#     url = f"{BASE}/{team_slug}/kader/verein/{team_id}/plus/0/galerie/0?saison_id={season}"
#     html = fetch_html(url)
#     if html is None:
#         return None
#     soup = BeautifulSoup(html, "lxml")
#     avg_age, value_mio, value_avg_mio = parse_tm_row_summary(soup)
#     squad_size, pct_extranjeros = parse_squad_table(soup)
#     return {
#         "Season": season,
#         "team_slug": team_slug,
#         "avg_age": avg_age,
#         "value_mio": value_mio,
#         "value_avg_mio": value_avg_mio,
#         "squad_size": squad_size,
#         "pct_foreigners": pct_extranjeros
#     }

# slug_map = {
#     "real-madrid":         ("real-madrid", 418),
#     "fc-barcelona":        ("fc-barcelona", 131),
#     "atletico-madrid":     ("atletico-madrid", 13),
#     "athletic-bilbao":     ("athletic-club", 621),
#     "sevilla-fc":          ("sevilla-fc", 368),
#     "valencia-cf":         ("valencia-cf", 1049),
#     "villarreal-cf":       ("villarreal-cf", 1050),
#     "real-sociedad":       ("real-sociedad", 681),
#     "real-betis":          ("real-betis", 150),
#     "ca-osasuna":          ("ca-osasuna", 331),
#     "espanyol-barcelona":  ("rcd-espanyol", 714),
#     "getafe-cf":           ("getafe-cf", 3709),
#     "rc-celta-de-vigo":    ("rc-celta-de-vigo", 940),
#     "rcd-mallorca":        ("rcd-mallorca", 237),
#     "ud-las-palmas":       ("ud-las-palmas", 472),
#     "cadiz-cf":            ("cadiz-cf", 2687),
#     "ud-almeria":          ("ud-almeria", 3302),
#     "granada-cf":          ("granada-cf", 16795),
#     "deportivo-alaves":    ("deportivo-alaves", 1108),
#     "levante-ud":          ("levante-ud", 3368),
#     "rayo-vallecano":      ("rayo-vallecano", 367),
#     "sd-eibar":            ("sd-eibar", 1533),
#     "girona-fc":           ("girona-fc", 12321),
#     "cd-leganes":          ("cd-leganes", 1244),
#     "sd-huesca":           ("sd-huesca", 5358),
#     "real-valladolid":     ("real-valladolid", 366),
#     "elche-cf":            ("elche-cf", 1531),
#     "cordoba-cf":          ("cordoba-cf", 993),
#     "real-sporting":       ("sporting-gijon", 2448),
#     "deportivo-la-coruna": ("deportivo-la-coruna", 897),
#     "real-zaragoza":       ("real-zaragoza", 142),
#     "xerez-cd":            ("xerez-cd", 134),
#     "cd-tenerife":         ("cd-tenerife", 648),
#     "recreativo-huelva":   ("recreativo-huelva", 2867),
#     "cd-numancia":         ("cd-numancia", 2296),
#     "real-murcia-cf":      ("real-murcia", 171),
#     "gimnastic-de-tarragona": ("gimnastic-tarragona", 5648),
#     "racing-santander":    ("racing-santander", 630),
#     "hercules-alicante":   ("hercules-cf", 7971),
#     "malaga-cf": ("malaga-cf", 1084)
# }

# seasons_map = {
#     "real-madrid": list(range(2005, 2025)),
#     "fc-barcelona": list(range(2005, 2025)),
#     "atletico-madrid": list(range(2005, 2025)),
#     "athletic-bilbao": list(range(2005, 2025)),
#     "sevilla-fc": list(range(2005, 2025)),
#     "valencia-cf": list(range(2005, 2025)),
#     "villarreal-cf": list(range(2005, 2025)),
#     "real-sociedad": [2005,2006,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024],
#     "real-betis": [2005,2006,2007,2008,2011,2012,2013,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024],
#     "ca-osasuna": [2005,2006,2007,2008,2009,2010,2011,2012,2013,2016,2019,2020,2021,2022,2023,2024],
#     "espanyol-barcelona": [2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2021,2022,2024],
#     "getafe-cf": [2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2017,2018,2019,2020,2021,2022,2023,2024],
#     "rc-celta-de-vigo": [2005,2006,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024],
#     "rcd-mallorca": [2005,2006,2007,2008,2009,2010,2011,2012,2019,2020,2021,2022,2023,2024],
#     "ud-las-palmas": [2015,2016,2017,2023,2024],
#     "cadiz-cf": [2005,2020,2021,2022,2023],
#     "ud-almeria": [2007,2008,2009,2010,2013,2014,2022,2023],
#     "granada-cf": [2011,2012,2013,2014,2015,2016,2019,2020,2021,2023],
#     "deportivo-alaves": [2005,2016,2017,2018,2019,2020,2021,2023,2024],
#     "levante-ud": [2006,2007,2010,2011,2012,2013,2014,2015,2017,2018,2019,2020,2021],
#     "rayo-vallecano": [2011,2012,2013,2014,2015,2018,2021,2022,2023,2024],
#     "sd-eibar": [2014,2015,2016,2017,2018,2019,2020],
#     "girona-fc": [2017,2018,2022,2023,2024],
#     "cd-leganes": [2016,2017,2018,2019,2024],
#     "sd-huesca": [2018,2020],
#     "real-valladolid": [2007,2008,2009,2012,2013,2018,2019,2020,2022,2024],
#     "elche-cf": [2013,2014,2020,2021,2022],
#     "cordoba-cf": [2014],
#     "real-sporting": [2008,2009,2010,2011,2015,2016],
#     "deportivo-la-coruna": [2005,2006,2007,2008,2009,2010,2012,2014,2015,2016,2017],
#     "real-zaragoza": [2005,2006,2007,2009,2010,2011,2012],
#     "xerez-cd": [2009],
#     "cd-tenerife": [2009],
#     "recreativo-huelva": [2006,2007,2008],
#     "cd-numancia": [2008],
#     "real-murcia-cf": [2007],
#     "gimnastic-de-tarragona": [2006],
#     "racing-santander": [2005,2006,2007,2008,2009,2010,2011],
#     "hercules-alicante": [2010],
#     "malaga-cf": [2005,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017]
# }

In [ ]:
# records = []
# failures = []

# for slug, (team_slug, team_id) in tqdm(slug_map.items(), desc="Clubs"):
#     for season in seasons_map.get(slug, []):
#         try:
#             rec = scrape_tm(team_slug, team_id, season)
#             if rec is not None and all(v is not None for v in [rec["avg_age"], rec["value_mio"], rec["value_avg_mio"], rec["squad_size"], rec["pct_foreigners"]]):
#                 rec["team_n"] = slug
#                 records.append(rec)
#                 print(f"OK {slug} {season}")
#             else:
#                 failures.append((slug, team_slug, team_id, season))
#                 print(f"FAILED {slug} {season}")
#             time.sleep(np.random.uniform(5, 12))
#         except Exception as e:
#             print(f"Skip {slug} {season} → {e}")
#             failures.append((slug, team_slug, team_id, season))
#             time.sleep(np.random.uniform(20, 35))

# tm_df = pd.DataFrame(records)
# tm_df.to_parquet('/content/drive/MyDrive/TFM/data/processed/transfermarkt_€€_2005_2025.parquet', index=False)
# print("🗸 Scraped rows:", len(tm_df))
# print("Years missed:", failures)

En caso de tener failures, habria que volver a descargarlos:

In [ ]:
# parquet_path = '/content/drive/MyDrive/TFM/data/processed/transfermarkt_€€_2005_2025.parquet'
# df_orig = pd.read_parquet(parquet_path)

# retries = []
# failures_retry = []

# for slug, team_slug, team_id, season in tqdm(failures, desc="Retry failures"):
#     try:
#         rec = scrape_tm(team_slug, team_id, season)
#         if rec is not None and all(v is not None for v in [rec["avg_age"], rec["value_mio"], rec["value_avg_mio"], rec["squad_size"], rec["pct_foreigners"]]):
#             rec["team_n"] = slug
#             retries.append(rec)
#             print(f"OK {slug} {season} (retry)")
#         else:
#             print(f"FAILED {slug} {season} (retry)")
#             failures_retry.append((slug, team_slug, team_id, season))
#         time.sleep(np.random.uniform(7, 16))
#     except Exception as e:
#         print(f"Skip {slug} {season} → {e}")
#         failures_retry.append((slug, team_slug, team_id, season))
#         time.sleep(np.random.uniform(25, 35))

# df_retries = pd.DataFrame(retries)

# if not df_retries.empty:
#     df_total = pd.concat([df_orig, df_retries], ignore_index=True)
#     df_total = df_total.drop_duplicates(subset=["Season", "team_slug"], keep="last")
#     df_total.to_parquet(parquet_path, index=False)
#     print(f"Guardado actualizado: {parquet_path} (total filas: {len(df_total)})")
# else:
#     print("No se recuperó ningún nuevo registro. Parquet no actualizado.")

# print("Fallos tras reintento:", failures_retry)
# print(f"Total de fallidos en este reintento: {len(failures_retry)}")

## Continuar aquí

In [ ]:
TRANSFER_PATH = PROC / "transfermarkt_eur_2005_2025.parquet"
transfermarkt = pd.read_parquet(TRANSFER_PATH)

In [ ]:
transfermarkt.head(20)

,Season,team_slug,avg_age,value_mio,value_avg_mio,squad_size,pct_foreigners,team_n
0,2005,real-madrid,25.4,281.60,7.82,36,36.11,real-madrid
1,2006,real-madrid,25.3,372.20,9.54,39,38.46,real-madrid
2,2007,real-madrid,26.4,355.80,13.68,26,65.38,real-madrid
3,2008,real-madrid,26.0,428.60,11.91,36,55.56,real-madrid
4,2009,real-madrid,25.7,451.70,14.12,32,50.00,real-madrid
5,2010,real-madrid,24.7,519.00,13.66,38,42.11,real-madrid
6,2011,real-madrid,25.2,539.20,16.34,33,48.48,real-madrid
7,2012,real-madrid,25.9,582.40,17.13,34,55.88,real-madrid
8,2013,real-madrid,25.5,636.80,19.90,32,46.88,real-madrid
9,2014,real-madrid,24.7,787.80,19.21,41,46.34,real-madrid


**Aqui habría que seguir el codigo en caso de querer añadir los datos de Transfermarkt para más temporadas**

In [ ]:
FD_XG_ELO_PATH = PROC / "fd_xg_elo_2005_2025.parquet"
fd_xg_elo = pd.read_parquet(FD_XG_ELO_PATH)
fd_xg_elo

,1XBA,1XBCA,1XBCD,1XBCH,1XBD,1XBH,AC,AF,AHCh,AHh,...,WHCD,WHCH,WHD,WHH,HomeTeam_norm,AwayTeam_norm,h_xg,a_xg,h_elo,a_elo
0,NaN,NaN,NaN,NaN,NaN,NaN,7,19,NaN,NaN,...,NaN,NaN,3.4,6.00,alaves,barcelona,<NA>,<NA>,1644.251709,1892.859375
1,NaN,NaN,NaN,NaN,NaN,NaN,4,19,NaN,NaN,...,NaN,NaN,3.0,1.90,ath bilbao,sociedad,<NA>,<NA>,1741.242554,1716.347778
2,NaN,NaN,NaN,NaN,NaN,NaN,5,14,NaN,NaN,...,NaN,NaN,3.1,1.95,valencia,betis,<NA>,<NA>,1804.846436,1812.068970
3,NaN,NaN,NaN,NaN,NaN,NaN,4,22,NaN,NaN,...,NaN,NaN,3.2,1.70,ath madrid,zaragoza,<NA>,<NA>,1734.815430,1718.908691
4,NaN,NaN,NaN,NaN,NaN,NaN,8,25,NaN,NaN,...,NaN,NaN,3.6,6.50,cadiz,real madrid,<NA>,<NA>,1659.813232,1887.151733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7735,NaN,NaN,NaN,NaN,NaN,NaN,2,18,0.0,0.00,...,NaN,NaN,NaN,NaN,sociedad,villarreal,1.3715,1.63038,1664.765259,1776.900146
7736,NaN,NaN,NaN,NaN,NaN,NaN,4,22,0.0,0.25,...,NaN,NaN,NaN,NaN,sevilla,betis,0.735196,1.17861,1635.653809,1748.659424
7737,NaN,NaN,NaN,NaN,NaN,NaN,3,15,-0.5,-0.50,...,NaN,NaN,NaN,NaN,celta,espanol,0.635572,0.599243,1681.137329,1646.712402
7738,NaN,NaN,NaN,NaN,NaN,NaN,5,13,1.5,1.25,...,NaN,NaN,NaN,NaN,girona,real madrid,1.05171,2.21359,1594.509766,1942.607300


Lo primero que hacemos para poder juntar las dos tablas será crear una variables Season que indique el año de la temporada a la que corresponde cada partido.

In [ ]:
fd_xg_elo['Date'] = pd.to_datetime(fd_xg_elo['Date'])

fd_xg_elo['Season'] = fd_xg_elo['Date'].apply(
    lambda x: x.year if x.month > 7 else x.year - 1
)

In [ ]:
team_norm_to_slug = {
    'alaves':        'deportivo-alaves',
    'ath bilbao':    'athletic-club',
    'valencia':      'valencia-cf',
    'ath madrid':    'atletico-madrid',
    'cadiz':         'cadiz-cf',
    'celta':         'rc-celta-de-vigo',
    'espanol':       'rcd-espanyol',
    'mallorca':      'rcd-mallorca',
    'osasuna':       'ca-osasuna',
    'sevilla':       'sevilla-fc',
    'real madrid':   'real-madrid',
    'betis':         'real-betis',
    'la coruna':     'deportivo-la-coruna',
    'barcelona':     'fc-barcelona',
    'getafe':        'getafe-cf',
    'malaga':        'malaga-cf',
    'santander':     'racing-santander',
    'sociedad':      'real-sociedad',
    'villarreal':    'villarreal-cf',
    'zaragoza':      'real-zaragoza',
    'recreativo':    'recreativo-huelva',
    'levante':       'levante-ud',
    'gimnastic':     'gimnastic-tarragona',
    'murcia':        'real-murcia',
    'almeria':       'ud-almeria',
    'valladolid':    'real-valladolid',
    'numancia':      'cd-numancia',
    'sp gijon':      'sporting-gijon',
    'tenerife':      'cd-tenerife',
    'xerez':         'xerez-cd',
    'hercules':      'hercules-cf',
    'granada':       'granada-cf',
    'vallecano':     'rayo-vallecano',
    'elche':         'elche-cf',
    'eibar':         'sd-eibar',
    'cordoba':       'cordoba-cf',
    'las palmas':    'ud-las-palmas',
    'leganes':       'cd-leganes',
    'girona':        'girona-fc',
    'huesca':        'sd-huesca'
}

In [ ]:
fd_xg_elo['home_team_slug'] = fd_xg_elo['HomeTeam_norm'].map(team_norm_to_slug)
fd_xg_elo['away_team_slug'] = fd_xg_elo['AwayTeam_norm'].map(team_norm_to_slug)

In [ ]:
fd_xg_elo = fd_xg_elo.merge(
    transfermarkt.rename(columns={
        'team_slug': 'home_team_slug',
        'avg_age': 'h_avg_age',
        'value_mio': 'h_value_mio',
        'value_avg_mio': 'h_value_avg_mio',
        'squad_size' : 'h_squad_size',
        'pct_foreigners' : 'h_pct_foreigners'
    })[['Season','home_team_slug','h_avg_age','h_value_mio','h_value_avg_mio', 'h_squad_size', 'h_pct_foreigners']],
    on=['Season','home_team_slug'],
    how='left'
)

In [ ]:
fd_xg_elo = fd_xg_elo.merge(
    transfermarkt.rename(columns={
        'team_slug': 'away_team_slug',
        'avg_age': 'a_avg_age',
        'value_mio': 'a_value_mio',
        'value_avg_mio': 'a_value_avg_mio',
        'squad_size' : 'a_squad_size',
        'pct_foreigners' : 'a_pct_foreigners'
    })[['Season','away_team_slug','a_avg_age','a_value_mio','a_value_avg_mio', 'a_squad_size', 'a_pct_foreigners']],
    on=['Season','away_team_slug'],
    how='left'
)

In [ ]:
fd_xg_elo

,1XBA,1XBCA,1XBCD,1XBCH,1XBD,1XBH,AC,AF,AHCh,AHh,...,h_avg_age,h_value_mio,h_value_avg_mio,h_squad_size,h_pct_foreigners,a_avg_age,a_value_mio,a_value_avg_mio,a_squad_size,a_pct_foreigners
0,NaN,NaN,NaN,NaN,NaN,NaN,7,19,NaN,NaN,...,28.2,34.830,1.120,31.0,54.84,25.4,327.50,9.63,34.0,47.06
1,NaN,NaN,NaN,NaN,NaN,NaN,4,19,NaN,NaN,...,25.2,47.230,1.150,41.0,2.44,25.9,53.83,1.74,31.0,22.58
2,NaN,NaN,NaN,NaN,NaN,NaN,5,14,NaN,NaN,...,27.3,213.550,6.280,34.0,41.18,26.2,85.95,2.60,33.0,24.24
3,NaN,NaN,NaN,NaN,NaN,NaN,4,22,NaN,NaN,...,24.2,134.150,4.330,31.0,25.81,27.7,66.55,2.66,25.0,28.00
4,NaN,NaN,NaN,NaN,NaN,NaN,8,25,NaN,NaN,...,28.8,2.215,0.791,28.0,46.43,25.4,281.60,7.82,36.0,36.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7735,NaN,NaN,NaN,NaN,NaN,NaN,2,18,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7736,NaN,NaN,NaN,NaN,NaN,NaN,4,22,0.0,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7737,NaN,NaN,NaN,NaN,NaN,NaN,3,15,-0.5,-0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7738,NaN,NaN,NaN,NaN,NaN,NaN,5,13,1.5,1.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
PROC.mkdir(parents=True, exist_ok=True)
SAVE_PATH = PROC / "fd_xg_elo_transfermarkt_2005_2025.parquet"

fd_xg_elo.to_parquet(SAVE_PATH, index=False)
print(f"Archivo guardado en: {SAVE_PATH}")

Archivo guardado en: data/02_processed/fd_xg_elo_transfermarkt_2005_2025.parquet


# Fbref (Jornadas)

In [ ]:
import os
os.environ["FOOTBALL_DATA_TOKEN"] = "4eb6e73bff904004aa14c2e96014a735"

In [ ]:
# ======================================================
#  ACTUALIZAR FECHAS DE wk_2005_2025.parquet (SOLO TEMPORADA DE RUN_DATE)
#  Fuente: football-data.org v4 (PD)
#  Guarda: wk_actualizado_2005_2025.parquet
#  Si falta FOOTBALL_DATA_TOKEN → se omite la actualización (no rompe).
# ======================================================

from pathlib import Path
from datetime import datetime
import time, os, requests, pandas as pd, numpy as np, unicodedata, re, pytz

# --- config y paths ---
TZ = pytz.timezone("Europe/Madrid")
PROC = Path(PROC) if "PROC" in globals() else Path("./data/02_processed")
WK_PATH_IN  = PROC / "wk_2005_2025.parquet"
WK_PATH_OUT = PROC / "wk_actualizado_2005_2025.parquet"

FD_BASE = "https://api.football-data.org/v4"
COMP = "PD"  # LaLiga

# --- helpers de normalización (canónicos) ---
def _strip_accents(s: str) -> str:
    t = unicodedata.normalize("NFKD", s or "")
    return "".join(c for c in t if not unicodedata.combining(c))

def _canon(s: str) -> str:
    s = _strip_accents(str(s)).lower()
    s = re.sub(r"[^a-z0-9]+"," ", s).strip()
    return s

def _season_from_run_date(run_date_str: str) -> int:
    d = pd.to_datetime(run_date_str)
    return int(d.year) if d.month >= 7 else int(d.year) - 1

def _utc_to_local_date_str(utc_iso: str, tz=TZ) -> str | None:
    try:
        dt_utc = pd.to_datetime(utc_iso, utc=True)
        dt_loc = dt_utc.tz_convert(tz)
        return dt_loc.date().strftime("%Y-%m-%d")
    except Exception:
        return None

# --- obtener token sin romper si falta ---
FD_TOKEN = ""
FD_TOKEN = globals().get("FOOTBALL_DATA_TOKEN", FD_TOKEN)
try:
    FD_TOKEN = (userdata.get("FOOTBALL_DATA_TOKEN") or FD_TOKEN)
except Exception:
    pass
FD_TOKEN = os.environ.get("FOOTBALL_DATA_TOKEN", FD_TOKEN)

if not FD_TOKEN:
    print("[FD] Sin FOOTBALL_DATA_TOKEN – omito la actualización del WK. (No se genera wk_actualizado_2005_2025.parquet)")
else:
    # --- carga parquet wk (ORIGINAL) ---
    wk = pd.read_parquet(WK_PATH_IN)
    # asumimos columnas: Season, Wk, Date, Home, Away
    for col in ["Season","Wk","Date","Home","Away"]:
        if col not in wk.columns:
            raise ValueError(f"El parquet WK no tiene la columna requerida: {col}")

    wk["Season"] = pd.to_numeric(wk["Season"], errors="coerce").astype("Int64")
    wk["Wk"]     = pd.to_numeric(wk["Wk"],     errors="coerce").astype("Int64")
    wk["Date"]   = pd.to_datetime(wk["Date"],  errors="coerce")
    wk = wk.dropna(subset=["Season","Wk"]).copy()
    wk["Season"] = wk["Season"].astype(int)
    wk["Wk"]     = wk["Wk"].astype(int)

    # columnas canónicas para emparejar con FD
    wk["home_canon"] = wk["Home"].map(_canon)
    wk["away_canon"] = wk["Away"].map(_canon)

    # --- temporada objetivo (derivada de RUN_DATE) ---
    SEASON_INT = _season_from_run_date(RUN_DATE)

    # --- descarga FD: todos los partidos de PD en esa temporada ---
    def fetch_fd_pd_season(token: str, season_int: int) -> pd.DataFrame:
        url = f"{FD_BASE}/competitions/{COMP}/matches"
        headers = {"X-Auth-Token": token}
        params = {"season": season_int}
        r = requests.get(url, headers=headers, params=params, timeout=30)
        if r.status_code == 429:  # rate limit simple backoff
            time.sleep(5)
            r = requests.get(url, headers=headers, params=params, timeout=30)
        r.raise_for_status()
        data = r.json() or {}
        matches = data.get("matches", []) or []
        rows = []
        for m in matches:
            rows.append({
                "season": season_int,
                "matchday": m.get("matchday"),
                "utcDate": m.get("utcDate"),
                "status": (m.get("status") or "").upper(),
                "home": (m.get("homeTeam") or {}).get("name"),
                "away": (m.get("awayTeam") or {}).get("name"),
            })
        df = pd.DataFrame(rows)
        if df.empty:
            return df
        df["Date"] = df["utcDate"].apply(_utc_to_local_date_str)
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        df["matchday"] = pd.to_numeric(df["matchday"], errors="coerce").astype("Int64")
        df["home_canon"] = df["home"].map(_canon)
        df["away_canon"] = df["away"].map(_canon)
        return df

    try:
        fd = fetch_fd_pd_season(FD_TOKEN, SEASON_INT)
    except Exception as e:
        print("[FD] Error al llamar a football-data.org → omito actualización:", e)
        fd = pd.DataFrame()

    if fd.empty:
        print(f"[FD] Sin datos para PD season {SEASON_INT}. No se actualiza WK.")
    else:
        # Solo FUTUROS respecto a RUN_DATE y estados programados
        run_dt = pd.to_datetime(RUN_DATE)
        fd_fut = fd[(fd["Date"].notna()) & (fd["Date"] >= run_dt)].copy()
        keep_status = {"SCHEDULED","TIMED","POSTPONED"}
        if "status" in fd_fut.columns:
            fd_fut = fd_fut[fd_fut["status"].isin(keep_status)].copy()

        # Preparar claves FD
        fd_fut["Season"] = SEASON_INT
        fd_fut["Wk_fd"]  = fd_fut["matchday"].astype("Int64")
        fd_small = fd_fut[["Season","Wk_fd","Date","home_canon","away_canon"]].copy()

        # Subconjunto del WK solo de esa Season
        wk_s = wk[wk["Season"] == SEASON_INT].copy()
        wk_s["idx"] = wk_s.index

        # Merge por Season + equipos
        merged = fd_small.merge(
            wk_s[["Season","Wk","Date","home_canon","away_canon","idx"]],
            on=["Season","home_canon","away_canon"],
            how="left",
            suffixes=("_fd","_wk")
        )

        updates = []
        not_found = 0

        def _choose_target_rows(sub):
            if sub["idx"].notna().sum() == 0:
                return pd.DataFrame(columns=sub.columns)
            # preferencia 1: coincidencia exacta de Wk
            exact = sub[(sub["Wk"].notna()) & (sub["Wk_fd"].notna()) & (sub["Wk"] == sub["Wk_fd"])]
            if len(exact):
                return exact
            # preferencia 2: Date_wk nula o >= RUN_DATE
            tmp = sub.copy()
            tmp["Date_wk"] = pd.to_datetime(tmp["Date_wk"], errors="coerce")
            pref = tmp[tmp["Date_wk"].isna() | (tmp["Date_wk"] >= run_dt)]
            if len(pref):
                return pref
            # preferencia 3: la más cercana futura por Date_wk
            return tmp.sort_values("Date_wk").head(1)

        # Aplicar por clave FD (Season+home+away)
        for key, g in merged.groupby(["Season","home_canon","away_canon"], dropna=False):
            sel = _choose_target_rows(g)
            if sel.empty:
                not_found += 1
                continue
            new_date = pd.to_datetime(sel["Date_fd"].iloc[0], errors="coerce")
            for _, r in sel.iterrows():
                idx = int(r["idx"])
                old_date = wk.at[idx, "Date"]
                if pd.isna(new_date):
                    continue
                if (pd.isna(old_date)) or (pd.to_datetime(old_date) != new_date):
                    wk.at[idx, "Date"] = new_date
                    updates.append({
                        "idx": idx,
                        "Season": int(r["Season"]),
                        "Home": wk.at[idx, "Home"],
                        "Away": wk.at[idx, "Away"],
                        "old_Date": pd.to_datetime(old_date).strftime("%Y-%m-%d") if pd.notna(old_date) else None,
                        "new_Date": new_date.strftime("%Y-%m-%d"),
                        "Wk": int(wk.at[idx, "Wk"]),
                        "Wk_fd": int(r["Wk_fd"]) if pd.notna(r["Wk_fd"]) else None
                    })

        # Guardar parquet ACTUALIZADO con el nuevo nombre
        wk_out = wk.drop(columns=["home_canon","away_canon"], errors="ignore").copy()
        wk_out.to_parquet(WK_PATH_OUT, index=False)

        print(f"[FD→WK] Season {SEASON_INT} | RUN_DATE {RUN_DATE}")
        print(f"         Partidos futuros FD: {len(fd_fut)} | Actualizaciones aplicadas: {len(updates)} | No encontrados: {not_found}")
        print(f"         Guardado parquet actualizado en: {WK_PATH_OUT}")

[FD→WK] Season 2025 | RUN_DATE 2025-12-02
         Partidos futuros FD: 238 | Actualizaciones aplicadas: 23 | No encontrados: 0
         Guardado parquet actualizado en: data/02_processed/wk_actualizado_2005_2025.parquet


In [ ]:
# # === wk_2005_2025.parquet (Season, Wk, Date, Home, Away) ===
# # Fuentes:
# #  - FBref + BeautifulSoup: 2005–2021
# #  - football-data.org (API v4): 2022–2025
# # Fallbacks: cache por temporada + manual CSV/HTML si hay rate-limit

# # -----------------------
# # Rutas / Config
# # -----------------------
# ROOT   = Path.cwd()
# PROC   = ROOT / "data" / "02_processed"
# CACHE  = ROOT / "data" / "external" / "fbref_cache"
# MANUAL = CACHE / "manual"
# for p in (PROC, CACHE, MANUAL):
#     p.mkdir(parents=True, exist_ok=True)

# WK_OUT = PROC / "wk_2005_2025.parquet"

# # -----------------------
# # FBref helpers
# # -----------------------
# def _fbref_urls(season_start: int) -> list[str]:
#     y1, y2 = season_start, season_start + 1
#     s = f"{y1}-{y2}"
#     return [
#         f"https://fbref.com/en/comps/12/{s}/schedule/{s}-La-Liga-Scores-and-Fixtures",
#         f"https://fbref.com/en/comps/12/{s}/schedule/La-Liga-Scores-and-Fixtures",
#         f"https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures-{s}",
#         "https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures",
#     ]

# def _http_get(url: str, session: requests.Session | None = None,
#               tries: int = 3, backoff: float = 1.5, jitter: float = 0.5,
#               deadline_ts: float | None = None) -> str | None:
#     sess = session or requests.Session()
#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
#         "Accept-Language": "en-US,en;q=0.9",
#         "Referer": "https://fbref.com/",
#         "Cache-Control": "no-cache",
#     }
#     for i in range(tries):
#         if deadline_ts is not None and time.time() >= deadline_ts:
#             return None
#         timeout = 20
#         if deadline_ts is not None:
#             timeout = max(3, min(timeout, int(deadline_ts - time.time())))
#         try:
#             r = sess.get(url, headers=headers, timeout=timeout)
#         except requests.Timeout:
#             continue

#         if r.status_code == 200 and len(r.text) > 8000:
#             return r.text

#         if r.status_code in (429, 503):
#             wait = backoff * (2 ** i) * (1 + random.uniform(0, jitter))
#             if deadline_ts is not None:
#                 wait = min(wait, max(0, deadline_ts - time.time()))
#             if wait > 0:
#                 time.sleep(wait)
#             continue

#         time.sleep(0.8)
#     return None

# def _parse_fbref_fixtures_table(html: str, season_start: int) -> pd.DataFrame:
#     soup = BeautifulSoup(html, "lxml")
#     # descomentar tablas en comentarios
#     for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
#         c.replace_with(c)

#     # 1) intenta por id exacto (suele existir): sched_<YYYY-YYYY>_12_1
#     table_id = f"sched_{season_start}-{season_start+1}_12_1"
#     dfs = []
#     table = soup.find("table", id=table_id)
#     if table is not None:
#         try:
#             dfs = pd.read_html(StringIO(str(table)))
#         except Exception:
#             dfs = []

#     # 2) si por id no sale, parsea todo el soup
#     if not dfs:
#         dfs = pd.read_html(StringIO(str(soup)))

#     pick = None
#     for t in dfs:
#         cols = [str(c).strip() for c in t.columns]
#         has_wk_date = ("Wk" in cols and "Date" in cols)
#         has_home    = any(re.fullmatch(r"\s*Home\s*", c) for c in cols)
#         has_away    = any(re.fullmatch(r"\s*Away\s*", c) for c in cols)
#         if has_wk_date and has_home and has_away:
#             pick = t
#             break

#     if pick is None:
#         return pd.DataFrame(columns=["Season","Wk","Date","Home","Away"])

#     df = pick.copy()
#     # quitar cabeceras repetidas integradas
#     if "Date" in df.columns:
#         df = df[df["Date"].astype(str).str.lower() != "date"]

#     home_col = next(c for c in df.columns if re.fullmatch(r"\s*Home\s*", str(c)))
#     away_col = next(c for c in df.columns if re.fullmatch(r"\s*Away\s*", str(c)))

#     out = pd.DataFrame({
#         "Season": season_start,
#         "Wk":     pd.to_numeric(df["Wk"], errors="coerce").astype("Int64"),
#         "Date":   pd.to_datetime(df["Date"], errors="coerce").dt.strftime("%Y-%m-%d"),
#         "Home":   df[home_col].astype(str),
#         "Away":   df[away_col].astype(str),
#     }).dropna(subset=["Date","Home","Away"])

#     out = out.drop_duplicates(["Season","Wk","Date","Home","Away"]) \
#              .sort_values(["Season","Wk","Date","Home","Away"]) \
#              .reset_index(drop=True)
#     return out

# def fbref_fixtures_one_season(season_start: int, session: requests.Session | None = None,
#                               verbose: bool = True, time_budget_sec: float = 35.0) -> pd.DataFrame:
#     """Cache → Web con deadline → Manual (CSV/HTML/TXT). Devuelve Season,Wk,Date,Home,Away."""
#     cache_csv = CACHE / f"fixtures_{season_start}.csv"
#     # 1) cache
#     if cache_csv.exists():
#         try:
#             df = pd.read_csv(cache_csv)
#             if not df.empty and {"Season","Wk","Date","Home","Away"} <= set(df.columns):
#                 if verbose: print(f"[cache]  {season_start}: {len(df)} filas")
#                 return df
#         except Exception:
#             pass
#     # 2) web
#     end_by = time.time() + time_budget_sec
#     sess = session or requests.Session()
#     for url in _fbref_urls(season_start):
#         html = _http_get(url, session=sess, deadline_ts=end_by)
#         if not html:
#             if time.time() >= end_by: break
#             continue
#         df = _parse_fbref_fixtures_table(html, season_start)
#         if not df.empty:
#             df.to_csv(cache_csv, index=False)
#             if verbose: print(f"[web]    {season_start}: {len(df)} filas  ← {url}")
#             return df
#         if time.time() >= end_by:
#             break
#     # 3) manual
#     for ext in (".csv", ".html", ".txt"):
#         man = MANUAL / f"fixtures_{season_start}{ext}"
#         if man.exists():
#             try:
#                 if ext == ".csv":
#                     mdf = pd.read_csv(man)
#                     cols = [str(c).strip() for c in mdf.columns]
#                     if {"Wk","Date","Home","Away"}.issubset(cols):
#                         df = pd.DataFrame({
#                             "Season": season_start,
#                             "Wk":     pd.to_numeric(mdf["Wk"], errors="coerce").astype("Int64"),
#                             "Date":   pd.to_datetime(mdf["Date"], errors="coerce").dt.strftime("%Y-%m-%d"),
#                             "Home":   mdf["Home"].astype(str),
#                             "Away":   mdf["Away"].astype(str),
#                         }).dropna(subset=["Date","Home","Away"])
#                     else:
#                         df = pd.DataFrame()
#                 else:
#                     html = man.read_text(encoding="utf-8")
#                     df = _parse_fbref_fixtures_table(html, season_start)
#                 if not df.empty:
#                     df.to_csv(cache_csv, index=False)
#                     if verbose: print(f"[manual] {season_start}: {len(df)} filas  ← {man.name}")
#                     return df
#             except Exception as e:
#                 if verbose: print(f"[manual] {season_start}: error {type(e).__name__}: {e}")
#     if verbose: print(f"[fbref]  {season_start}: pendiente (sin datos web/manual).")
#     return pd.DataFrame(columns=["Season","Wk","Date","Home","Away"])

# # -----------------------
# # football-data.org helpers (2022–2025)
# # -----------------------
# def fdorg_read_matches_one_season(season_start: int, token: str,
#                                   verbose: bool = True) -> pd.DataFrame:
#     """
#     Devuelve Season, Date, Home, Away, Wk (matchday) para LaLiga (competition=PD).
#     """
#     url = f"https://api.football-data.org/v4/competitions/PD/matches?season={season_start}"
#     headers = {"X-Auth-Token": token}
#     r = requests.get(url, headers=headers, timeout=30)
#     # manejar 403/errores con mensaje claro
#     if r.status_code == 403:
#         raise RuntimeError(f"FD 403 (sin acceso a {season_start}).")
#     r.raise_for_status()
#     js = r.json()
#     rows = []
#     for m in js.get("matches", []):
#         rows.append({
#             "Season": season_start,
#             "Date": pd.to_datetime(m.get("utcDate"), errors="coerce").strftime("%Y-%m-%d"),
#             "Home": (m.get("homeTeam") or {}).get("name"),
#             "Away": (m.get("awayTeam") or {}).get("name"),
#             "Wk":   m.get("matchday"),
#         })
#     df = pd.DataFrame(rows).dropna(subset=["Date","Home","Away"])
#     if not df.empty:
#         df["Wk"] = pd.to_numeric(df["Wk"], errors="coerce").astype("Int64")
#         df = df.drop_duplicates(["Season","Wk","Date","Home","Away"]) \
#                .sort_values(["Season","Wk","Date","Home","Away"]).reset_index(drop=True)
#     if verbose: print(f"[fd.org] {season_start}: {len(df)} filas")
#     return df

# # -----------------------
# # Orquestador: 2005–2021 FBref, 2022–2025 football-data.org (con fallbacks)
# # -----------------------
# def build_wk_parquet_mixed(fbref_from=2005, fbref_to=2021, fd_from=2022, fd_to=2025,
#                            per_season_budget_sec=35.0, polite_pause_sec=0.8) -> pd.DataFrame:
#     frames = []
#     session = requests.Session()

#     # 1) FBref 2005..2021
#     for ss in range(fbref_from, fbref_to + 1):
#         df_ss = fbref_fixtures_one_season(ss, session=session, verbose=True,
#                                           time_budget_sec=per_season_budget_sec)
#         if df_ss.empty:
#             print(f"[WARN] {ss} sin datos FBref (añade manual CSV/HTML en {MANUAL} y reintenta).")
#         else:
#             frames.append(df_ss)
#         time.sleep(polite_pause_sec)

#     # 2) football-data.org 2022..2025 (con fallback FBref si 403)
#     token = os.environ.get("FOOTBALL_DATA_TOKEN") or os.environ.get("FOOTBALL_DATA_API_KEY")
#     if token is None:
#         print("[INFO] FOOTBALL_DATA_TOKEN no definido → intentar FBref también para 2022–2025.")
#     for ss in range(fd_from, fd_to + 1):
#         df_ss = pd.DataFrame()
#         if token:
#             try:
#                 df_ss = fdorg_read_matches_one_season(ss, token=token, verbose=True)
#                 # politeness para la API free
#                 time.sleep(2.0)
#             except Exception as e:
#                 print(f"[fd.org] {ss}: {e} → fallback FBref")
#         if df_ss.empty:
#             fb_df = fbref_fixtures_one_season(ss, session=session, verbose=True,
#                                               time_budget_sec=per_season_budget_sec)
#             if fb_df.empty:
#                 print(f"[WARN] {ss} sigue vacío (usa manual fixtures_{ss}.csv/.html en {MANUAL}).")
#             else:
#                 df_ss = fb_df
#         if not df_ss.empty:
#             frames.append(df_ss)
#         time.sleep(polite_pause_sec)

#     # 3) Unión y guardado
#     wk = (pd.concat(frames, ignore_index=True)
#           if frames else pd.DataFrame(columns=["Season","Wk","Date","Home","Away"]))
#     wk.to_parquet(WK_OUT, index=False)

#     # 4) Resumen
#     print("\nGuardado:", WK_OUT, "· filas =", len(wk))
#     print("Partidos por temporada:")
#     if not wk.empty:
#         display(wk.groupby("Season").size().rename("n_matches").to_frame())

#     # 5) Validación suave (opcional)
#     expected = list(range(fbref_from, fd_to + 1))
#     missing = [s for s in expected if s not in set(wk["Season"])]
#     if missing:
#         print("Temporadas faltantes:", missing, "→ revisa logs / añade manual y reejecuta.")
#     else:
#         print("Cobertura completa de temporadas.")
#     return wk

# # --- Ejecutar ---
# wk_all = build_wk_parquet_mixed(2005, 2021, 2022, 2025)

In [ ]:
# ============================================================
# Inyectar Matchweek en el parquet principal
# Lee:  data/02_processed/wk_actualizado_2005_2025.parquet  (Season, Wk, Date, Home, Away)
# Lee:  data/02_processed/fd_xg_elo_2005_2025.parquet  (tu consolidado)
# Une por: Season, Date, HomeTeam_norm, AwayTeam_norm (normalizados)
# Aplica fuzzy matching (±2 días) + pase 2011 (±10 días) + pase por pareja + parche manual
# Escribe: data/02_processed/fd_xg_elo_wk_2005_2025.parquet
# ============================================================

# -----------------------
# Rutas
# -----------------------
ROOT = Path.cwd()
PROC = ROOT / "data" / "02_processed"

# ⬇️ AJUSTA el nombre si tu parquet principal se llama distinto
MAIN_IN  = PROC / "fd_xg_elo_transfermarkt_2005_2025.parquet"
WK_PARQ  = PROC / "wk_actualizado_2005_2025.parquet"
MAIN_OUT = PROC / "fd_xg_elo_transfermarkt_wk_2005_2025.parquet"

# -----------------------
# Normalizador + mapeos
# -----------------------
def _norm_text(s: str) -> str:
    s = "" if s is None else str(s)
    t = unicodedata.normalize("NFKD", s)
    t = "".join(c for c in t if not unicodedata.combining(c))
    t = re.sub(r"[^A-Za-z0-9]+", " ", t).strip().lower()
    return t

# Variantes → clave canónica (incluye Espanyol y Oviedo ampliados)
FBREF_TO_NORM = {
    # Alavés
    "alaves":"alaves","deportivo alaves":"alaves","deportivo de alaves":"alaves","alaves cf":"alaves","alaves club":"alaves",
    "alaves s a d":"alaves","alavés":"alaves","deportivo alavés":"alaves",
    # Athletic Club
    "athletic club":"ath bilbao","athletic bilbao":"ath bilbao","ath bilbao":"ath bilbao","bilbao":"ath bilbao",
    # Valencia
    "valencia":"valencia","valencia cf":"valencia",
    # Atlético de Madrid
    "atletico madrid":"ath madrid","atlético madrid":"ath madrid","atletico de madrid":"ath madrid",
    "club atletico de madrid":"ath madrid","atl madrid":"ath madrid",
    # Cádiz
    "cadiz":"cadiz","cádiz":"cadiz","cadiz cf":"cadiz","cádiz cf":"cadiz",
    # Celta
    "celta":"celta","celta vigo":"celta","rc celta de vigo":"celta","rc celta vigo":"celta","celta de vigo":"celta",
    # Espanyol (todas las variantes habituales)
    "espanyol":"espanol","rcd espanyol":"espanol","r c d espanyol":"espanol",
    "espanyol barcelona":"espanol","espanyol de barcelona":"espanol","rcd espanyol de barcelona":"espanol",
    # Mallorca
    "mallorca":"mallorca","rcd mallorca":"mallorca",
    # Osasuna
    "osasuna":"osasuna","ca osasuna":"osasuna",
    # Sevilla
    "sevilla":"sevilla","sevilla fc":"sevilla",
    # Real Madrid
    "real madrid":"real madrid","real madrid cf":"real madrid",
    # Betis
    "betis":"betis","real betis":"betis","real betis balompie":"betis","real betis balompié":"betis",
    # Deportivo La Coruña
    "deportivo la coruna":"la coruna","deportivo la coruña":"la coruna","rc deportivo la coruna":"la coruna",
    "rc deportivo la coruña":"la coruna","deportivo":"la coruna",
    # Barcelona
    "barcelona":"barcelona","fc barcelona":"barcelona","barça":"barcelona",
    # Getafe
    "getafe":"getafe","getafe cf":"getafe",
    # Málaga
    "malaga":"malaga","málaga":"malaga","malaga cf":"malaga","málaga cf":"malaga",
    # Racing Santander
    "racing sant":"santander","santander":"santander","real racing club":"santander",
    # Real Sociedad
    "real sociedad":"sociedad","sociedad":"sociedad","real sociedad de futbol":"sociedad","real sociedad de fútbol":"sociedad",
    # Villarreal
    "villarreal":"villarreal","villarreal cf":"villarreal",
    # Zaragoza
    "zaragoza":"zaragoza","real zaragoza":"zaragoza",
    # Recreativo
    "recreativo":"recreativo","recreativo huelva":"recreativo",
    # Levante
    "levante":"levante","levante ud":"levante",
    # Gimnàstic Tarragona
    "gimnastic":"gimnastic","gimnastic tarragona":"gimnastic","gimnàstic tarragona":"gimnastic","gimnàstic":"gimnastic",
    # Murcia
    "murcia":"murcia","real murcia":"murcia",
    # Almería
    "almeria":"almeria","almería":"almeria","ud almeria":"almeria",
    # Valladolid
    "valladolid":"valladolid","real valladolid":"valladolid",
    # Numancia
    "numancia":"numancia","cd numancia":"numancia",
    # Sporting Gijón
    "sporting gijon":"sp gijon","sporting de gijon":"sp gijon","real sporting de gijon":"sp gijon",
    "sp gijon":"sp gijon","real sporting":"sp gijon",
    # Tenerife
    "tenerife":"tenerife","cd tenerife":"tenerife",
    # Xerez
    "xerez":"xerez","xerez cd":"xerez",
    # Hércules
    "hercules":"hercules","hércules":"hercules","hercules cf":"hercules",
    # Granada
    "granada":"granada","granada cf":"granada",
    # Rayo Vallecano
    "rayo vallecano":"vallecano","vallecano":"vallecano","rayo vallecano de madrid":"vallecano",
    # Elche
    "elche":"elche","elche cf":"elche",
    # Eibar
    "eibar":"eibar","sd eibar":"eibar",
    # Córdoba
    "cordoba":"cordoba","córdoba":"cordoba","cordoba cf":"cordoba","córdoba cf":"cordoba",
    # Las Palmas
    "las palmas":"las palmas","ud las palmas":"las palmas",
    # Leganés
    "leganes":"leganes","leganés":"leganes","cd leganes":"leganes",
    # Girona
    "girona":"girona","girona fc":"girona",
    # Huesca
    "huesca":"huesca","sd huesca":"huesca",
    # Oviedo (FD.org/otros)
    "real oviedo":"real oviedo","oviedo":"real oviedo","real oviedo cf":"real oviedo",
}

# Limpieza genérica para FD.org (sufijos/tokens)
STOP_TOKENS = {"cf","fc","ud","sd","cd","rcd","rc","s","sad"}
PHRASE_PATTERNS = [
    (r"\bde madrid\b$", ""),
    (r"\bde barcelona\b$", ""),
    (r"\bde futbol\b$", ""),
    (r"\bde f[úu]tbol\b$", ""),
    (r"\bs\.?a\.?d\.?\b", ""),
]

def _canon_team(name: str) -> str:
    base = _norm_text(name)
    # 1) diccionario directo
    if base in FBREF_TO_NORM:
        return FBREF_TO_NORM[base]
    # 2) limpieza de frases
    s = base
    for pat, repl in PHRASE_PATTERNS:
        s = re.sub(pat, repl, s).strip()
    # 3) elimina tokens clubísticos
    toks = [t for t in s.split() if t not in STOP_TOKENS]
    s2 = " ".join(toks).strip()
    # 4) fallback
    return FBREF_TO_NORM.get(s2, s2)

# -----------------------
# Cargar WK y preparar clave
# -----------------------
wk = pd.read_parquet(WK_PARQ).copy()
if wk.empty:
    raise FileNotFoundError(f"WK vacío o no encontrado: {WK_PARQ}")

wk["Season"] = pd.to_numeric(wk["Season"], errors="coerce").astype("Int64")
wk["Date"]   = pd.to_datetime(wk["Date"], errors="coerce").dt.strftime("%Y-%m-%d")
wk = wk.rename(columns={"Wk": "Matchweek"})

wk["HomeTeam_norm"] = wk["Home"].map(_canon_team)
wk["AwayTeam_norm"] = wk["Away"].map(_canon_team)

wk_key = (wk[["Season","Date","HomeTeam_norm","AwayTeam_norm","Matchweek"]]
          .dropna(subset=["Date","HomeTeam_norm","AwayTeam_norm"])
          .drop_duplicates())

# -----------------------
# Cargar DF principal y alinear
# -----------------------
df = pd.read_parquet(MAIN_IN).copy()
if df.empty:
    raise FileNotFoundError(f"Principal vacío o no encontrado: {MAIN_IN}")

need = ["Season","Date","HomeTeam_norm","AwayTeam_norm"]
missing = [c for c in need if c not in df.columns]
if missing:
    raise ValueError(f"Faltan columnas en DF principal para merge: {missing}")

df["Season"] = pd.to_numeric(df["Season"], errors="coerce").astype("Int64")
df["Date"]   = pd.to_datetime(df["Date"], errors="coerce").dt.strftime("%Y-%m-%d")
df["HomeTeam_norm"] = df["HomeTeam_norm"].astype(str).map(_canon_team)
df["AwayTeam_norm"] = df["AwayTeam_norm"].astype(str).map(_canon_team)

# -----------------------
# Merge exacto
# -----------------------
merged = df.merge(
    wk_key,
    on=["Season","Date","HomeTeam_norm","AwayTeam_norm"],
    how="left",
    validate="m:1"
)
print(f"[WK] merge exacto: {len(merged)} filas; sin jornada={int(merged['Matchweek'].isna().sum())}")

# -----------------------
# Fuzzy por fecha (±2 días) y pase 2011 (±10 días)
# -----------------------
def _fuzzy_fill(merged: pd.DataFrame, wk_key: pd.DataFrame, max_days=2, season_mask=None):
    msk = merged["Matchweek"].isna()
    if season_mask is not None:
        msk &= season_mask
    if not msk.any():
        return merged

    left = merged.loc[msk, ["Season","Date","HomeTeam_norm","AwayTeam_norm"]].copy()
    left["Date_dt"] = pd.to_datetime(left["Date"])
    left["pair"] = left["HomeTeam_norm"] + "|" + left["AwayTeam_norm"]

    right = wk_key.copy()
    right["Date_dt"] = pd.to_datetime(right["Date"])
    right["pair"] = right["HomeTeam_norm"] + "|" + right["AwayTeam_norm"]

    cand = left.merge(
        right[["Season","pair","Date","Date_dt","Matchweek"]],
        on=["Season","pair"], how="left"
    )
    cand["absdiff"] = (cand["Date_dt_x"] - cand["Date_dt_y"]).abs().dt.days
    cand = cand[cand["absdiff"] <= max_days]

    if cand.empty:
        return merged

    idxmin = cand.groupby(["Season","pair","Date_x"], sort=False)["absdiff"].idxmin()
    best = cand.loc[idxmin, ["Season","pair","Date_x","Matchweek"]].rename(columns={"Date_x":"Date"})

    merged = merged.copy()
    merged["pair"] = merged["HomeTeam_norm"] + "|" + merged["AwayTeam_norm"]
    merged = merged.merge(best, on=["Season","pair","Date"], how="left", suffixes=("", "_fuzzy"))
    merged["Matchweek"] = merged["Matchweek"].fillna(merged["Matchweek_fuzzy"])
    merged = merged.drop(columns=["pair","Matchweek_fuzzy"])
    return merged

# 1º pase general ±2 días
merged = _fuzzy_fill(merged, wk_key, max_days=2)

# 2º pase solo temporada 2011 ±10 días (reprogramados)
if merged["Matchweek"].isna().any():
    merged = _fuzzy_fill(merged, wk_key, max_days=10, season_mask=(merged["Season"] == 2011))

print(f"[WK] tras fuzzy: sin jornada={int(merged['Matchweek'].isna().sum())}")

# -----------------------
# Tercer pase (FIX): por pareja exacta cuando hay una única jornada posible
# -----------------------
if merged["Matchweek"].isna().any():
    # 1) parejas Season+Home+Away con UNA única jornada en wk_key
    wk_counts = (wk_key
                 .groupby(["Season","HomeTeam_norm","AwayTeam_norm"])["Matchweek"]
                 .nunique()
                 .reset_index(name="n"))
    wk_one_pairs = wk_counts.loc[wk_counts["n"] == 1, ["Season","HomeTeam_norm","AwayTeam_norm"]]

    if not wk_one_pairs.empty:
        # 2) obtener el valor de esa única jornada por pareja
        wk_one_map = (wk_key
                      .merge(wk_one_pairs, on=["Season","HomeTeam_norm","AwayTeam_norm"], how="inner")
                      .groupby(["Season","HomeTeam_norm","AwayTeam_norm"], as_index=False)["Matchweek"]
                      .first()  # la única
                      )

        # 3) mergear el valor y rellenar
        before_na = int(merged["Matchweek"].isna().sum())
        merged = merged.merge(
            wk_one_map,
            on=["Season","HomeTeam_norm","AwayTeam_norm"],
            how="left",
            suffixes=("", "_pairval")
        )
        if "Matchweek_pairval" in merged.columns:
            merged["Matchweek"] = merged["Matchweek"].fillna(merged["Matchweek_pairval"])
            merged = merged.drop(columns=["Matchweek_pairval"])
        after_na = int(merged["Matchweek"].isna().sum())
        print(f"[WK] pase por pareja: rellenadas {before_na - after_na} filas; sin jornada={after_na}")
    else:
        print("[WK] pase por pareja: no hay parejas con jornada única.")
else:
    print("[WK] pase por pareja: no necesario (no hay NA).")


# -----------------------
# Parche manual explícito (caso documentado)
# Granada–Mallorca (2011-12-07) → Jornada 13
# -----------------------
MANUAL_WK = {
    (2011, "2011-12-07", "granada", "mallorca"): 13
}
for (yy, dd, hh, aa), wknum in MANUAL_WK.items():
    m = (merged["Season"].eq(yy) &
         merged["Date"].eq(dd) &
         merged["HomeTeam_norm"].eq(hh) &
         merged["AwayTeam_norm"].eq(aa) &
         merged["Matchweek"].isna())
    merged.loc[m, "Matchweek"] = wknum

print(f"[WK] tras parche manual: sin jornada={int(merged['Matchweek'].isna().sum())}")

# -----------------------
# Guardar y diagnóstico
# -----------------------
merged.to_parquet(MAIN_OUT, index=False)
print("✅ Guardado parquet con jornada →", MAIN_OUT)

# % con jornada por temporada
try:
    pct = merged.groupby("Season")["Matchweek"].apply(lambda s: (s.notna().mean()*100)).round(1)
    print("\n% filas con Matchweek por temporada:")
    print(pct.to_frame("%with_wk"))
except Exception:
    pass

# Exporta no emparejados (si quedara alguno)
unmatched = (merged[merged["Matchweek"].isna()]
             .sort_values(["Season","Date","HomeTeam_norm","AwayTeam_norm"]))
if len(unmatched):
    out_csv = PROC / "unmatched_without_matchweek.csv"
    unmatched[["Season","Date","HomeTeam_norm","AwayTeam_norm"]].to_csv(out_csv, index=False)
    print(f"⚠️ Aún quedan {len(unmatched)} sin jornada. Exportado a: {out_csv}")
else:
    print("🎯 Emparejado 100%.")

[WK] merge exacto: 7740 filas; sin jornada=34
[WK] tras fuzzy: sin jornada=1
[WK] pase por pareja: rellenadas 1 filas; sin jornada=0
[WK] tras parche manual: sin jornada=0
✅ Guardado parquet con jornada → /content/data/02_processed/fd_xg_elo_transfermarkt_wk_2005_2025.parquet

% filas con Matchweek por temporada:
        %with_wk
Season          
2005       100.0
2006       100.0
2007       100.0
2008       100.0
2009       100.0
2010       100.0
2011       100.0
2012       100.0
2013       100.0
2014       100.0
2015       100.0
2016       100.0
2017       100.0
2018       100.0
2019       100.0
2020       100.0
2021       100.0
2022       100.0
2023       100.0
2024       100.0
2025       100.0
🎯 Emparejado 100%.
